In [1]:
%pip install azure-search-documents==11.4.0b3
%pip show azure-search-documents
%pip install python-dotenv

import os
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient 
from azure.search.documents import SearchClient
from azure.search.documents.indexes.models import (  
    SearchIndex,  
    SearchField,  
    SearchFieldDataType,  
    SimpleField,  
    SearchableField,
    ComplexField,
    SearchIndex,  
    SemanticConfiguration,  
    PrioritizedFields,  
    SemanticField,  
    SemanticSettings,  
)

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Name: azure-search-documents
Version: 11.4.0b3
Summary: Microsoft Azure Cognitive Search Client Library for Python
Home-page: https://github.com/Azure/azure-sdk-for-python/tree/main/sdk/search/azure-search-documents
Author: Microsoft Corporation
Author-email: ascl@microsoft.com
License: MIT License
Location: c:\Users\210588\AppData\Local\Programs\Python\Python312\Lib\site-packages
Requires: azure-common, azure-core, isodate
Required-by: 
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


In [2]:
service_name = "sotuken-beta"
admin_key = "RvICL223LXinONIZB3GHMoQTP5MfmCUmoH4iav37B3AzSeA918fw"

index_name = "book-search"

endpoint = "https://sotuken-beta.search.windows.net".format(service_name)
admin_client = SearchIndexClient(endpoint=endpoint,
                      index_name=index_name,
                      credential=AzureKeyCredential(admin_key))

search_client = SearchClient(endpoint=endpoint,
                      index_name=index_name,
                      credential=AzureKeyCredential(admin_key))

In [27]:
try:
    result = admin_client.delete_index(index_name)
    print ('Index', index_name, 'Deleted')
except Exception as ex:
    print (ex)

Index book-search Deleted


In [28]:
name = "book-search"
fields = [
        SimpleField(name="BookId", type=SearchFieldDataType.String, key=True),
        SearchableField(name="BookName", type=SearchFieldDataType.String, sortable=True),
        SearchableField(name="BookAuthor", type=SearchFieldDataType.String, sortable=True),
        SearchableField(name="BookGenre", type=SearchFieldDataType.String, facetable=True, filterable=True, sortable=True),
        SearchableField(name="Release_Date", type=SearchFieldDataType.DateTimeOffset, facetable=True, filterable=True, sortable=True),

        SimpleField(name="Price", type=SearchFieldDataType.Int32, facetable=True, filterable=True, sortable=True),
        SimpleField(name="Rating", type=SearchFieldDataType.Double, facetable=True, filterable=True, sortable=True),

        SearchableField(name="Text", type=SearchFieldDataType.String, analyzer_name="en.lucene"),
        SearchableField(name="Outher_Keyword", type=SearchFieldDataType.String, analyzer_name="fr.lucene"),
]
semantic_config = SemanticConfiguration(
    name="my-semantic-config",
    prioritized_fields=PrioritizedFields(
        title_field=SemanticField(field_name="BookName"),
        prioritized_content_fields=[SemanticField(field_name="Text")],
        prioritized_keywords_fields=[SemanticField(field_name="BookGenre")]
    )
)

semantic_settings = SemanticSettings(configurations=[semantic_config])
scoring_profiles = []

In [29]:
index = SearchIndex(
    name=name,
    fields=fields,
    semantic_settings=semantic_settings,
    scoring_profiles=scoring_profiles,
)

try:
    result = admin_client.create_index(index)
    print ('Index', result.name, 'created')
except Exception as ex:
    print (ex)

Index book-search created


In [30]:
documents = [
    {
    "@search.action": "upload",
    "BookId": "92",
    "BookName": "蜘蛛の糸",
    "BookAuthor": "芥川龍之介",
    "BookGenre": "掌編小説",
    "Release_Date": "1918年7月",
    "Rating": "3.71",
    "Text": """ある日の事でございます｡御釈迦様は極楽の蓮池のふちを､独りでぶらぶら御歩きになっていらっしゃいました｡池の中に咲いている蓮の花は､みんな玉のようにまっ白で､そのまん中にある金色の蕊からは､何とも云えない好い匂が､絶間なくあたりへ溢れて居ります｡極楽は丁度朝なのでございましょう｡
 やがて御釈迦様はその池のふちに御佇みになって､水の面を蔽っている蓮の葉の間から､ふと下の容子を御覧になりました｡この極楽の蓮池の下は､丁度地獄の底に当って居りますから､水晶のような水を透き徹して､三途の河や針の山の景色が､丁度覗き眼鏡を見るように､はっきりと見えるのでございます｡
 するとその地獄の底に､ｶﾝﾀﾞﾀと云う男が一人､ほかの罪人と一しょに蠢いている姿が､御眼に止まりました｡このｶﾝﾀﾞﾀと云う男は､人を殺したり家に火をつけたり､いろいろ悪事を働いた大泥坊でございますが､それでもたった一つ､善い事を致した覚えがございます｡と申しますのは､ある時この男が深い林の中を通りますと､小さな蜘蛛が一匹､路ばたを這って行くのが見えました｡そこでｶﾝﾀﾞﾀは早速足を挙げて､踏み殺そうと致しましたが､｢いや､いや､これも小さいながら､命のあるものに違いない｡その命を無暗にとると云う事は､いくら何でも可哀そうだ｡｣と､こう急に思い返して､とうとうその蜘蛛を殺さずに助けてやったからでございます｡
 御釈迦様は地獄の容子を御覧になりながら､このｶﾝﾀﾞﾀには蜘蛛を助けた事があるのを御思い出しになりました｡そうしてそれだけの善い事をした報には､出来るなら､この男を地獄から救い出してやろうと御考えになりました｡幸い､側を見ますと､翡翠のような色をした蓮の葉の上に､極楽の蜘蛛が一匹､美しい銀色の糸をかけて居ります｡御釈迦様はその蜘蛛の糸をそっと御手に御取りになって､玉のような白蓮の間から､遥か下にある地獄の底へ､まっすぐにそれを御下しなさいました｡

二

 こちらは地獄の底の血の池で､ほかの罪人と一しょに､浮いたり沈んだりしていたｶﾝﾀﾞﾀでございます｡何しろどちらを見ても､まっ暗で､たまにそのくら暗からぼんやり浮き上っているものがあると思いますと､それは恐しい針の山の針が光るのでございますから､その心細さと云ったらございません｡その上あたりは墓の中のようにしんと静まり返って､たまに聞えるものと云っては､ただ罪人がつく微な嘆息ばかりでございます｡これはここへ落ちて来るほどの人間は､もうさまざまな地獄の責苦に疲れはてて､泣声を出す力さえなくなっているのでございましょう｡ですからさすが大泥坊のｶﾝﾀﾞﾀも､やはり血の池の血に咽びながら､まるで死にかかった蛙のように､ただもがいてばかり居りました｡
 ところがある時の事でございます｡何気なくｶﾝﾀﾞﾀが頭を挙げて､血の池の空を眺めますと､そのひっそりとした暗の中を､遠い遠い天上から､銀色の蜘蛛の糸が､まるで人目にかかるのを恐れるように､一すじ細く光りながら､するすると自分の上へ垂れて参るのではございませんか｡ｶﾝﾀﾞﾀはこれを見ると､思わず手を拍って喜びました｡この糸に縋りついて､どこまでものぼって行けば､きっと地獄からぬけ出せるのに相違ございません｡いや､うまく行くと､極楽へはいる事さえも出来ましょう｡そうすれば､もう針の山へ追い上げられる事もなくなれば､血の池に沈められる事もある筈はございません｡
 こう思いましたからｶﾝﾀﾞﾀは､早速その蜘蛛の糸を両手でしっかりとつかみながら､一生懸命に上へ上へとたぐりのぼり始めました｡元より大泥坊の事でございますから､こう云う事には昔から､慣れ切っているのでございます｡
 しかし地獄と極楽との間は､何万里となくございますから､いくら焦って見た所で､容易に上へは出られません｡ややしばらくのぼる中に､とうとうｶﾝﾀﾞﾀもくたびれて､もう一たぐりも上の方へはのぼれなくなってしまいました｡そこで仕方がございませんから､まず一休み休むつもりで､糸の中途にぶら下りながら､遥かに目の下を見下しました｡
 すると､一生懸命にのぼった甲斐があって､さっきまで自分がいた血の池は､今ではもう暗の底にいつの間にかかくれて居ります｡それからあのぼんやり光っている恐しい針の山も､足の下になってしまいました｡この分でのぼって行けば､地獄からぬけ出すのも､存外わけがないかも知れません｡ｶﾝﾀﾞﾀは両手を蜘蛛の糸にからみながら､ここへ来てから何年にも出した事のない声で､｢しめた｡しめた｡｣と笑いました｡ところがふと気がつきますと､蜘蛛の糸の下の方には､数限もない罪人たちが､自分ののぼった後をつけて､まるで蟻の行列のように､やはり上へ上へ一心によじのぼって来るではございませんか｡ｶﾝﾀﾞﾀはこれを見ると､驚いたのと恐しいのとで､しばらくはただ､莫迦のように大きな口を開いたまま､眼ばかり動かして居りました｡自分一人でさえ断れそうな､この細い蜘蛛の糸が､どうしてあれだけの人数の重みに堪える事が出来ましょう｡もし万一途中で断れたと致しましたら､折角ここへまでのぼって来たこの肝腎な自分までも､元の地獄へ逆落しに落ちてしまわなければなりません｡そんな事があったら､大変でございます｡が､そう云う中にも､罪人たちは何百となく何千となく､まっ暗な血の池の底から､うようよと這い上って､細く光っている蜘蛛の糸を､一列になりながら､せっせとのぼって参ります｡今の中にどうかしなければ､糸はまん中から二つに断れて､落ちてしまうのに違いありません｡
 そこでｶﾝﾀﾞﾀは大きな声を出して､｢こら､罪人ども｡この蜘蛛の糸は己のものだぞ｡お前たちは一体誰に尋いて､のぼって来た｡下りろ｡下りろ｡｣と喚きました｡
 その途端でございます｡今まで何ともなかった蜘蛛の糸が､急にｶﾝﾀﾞﾀのぶら下っている所から､ぷつりと音を立てて断れました｡ですからｶﾝﾀﾞﾀもたまりません｡あっと云う間もなく風を切って､独楽のようにくるくるまわりながら､見る見る中に暗の底へ､まっさかさまに落ちてしまいました｡
 後にはただ極楽の蜘蛛の糸が､きらきらと細く光りながら､月も星もない空の中途に､短く垂れているばかりでございます｡

三

 御釈迦様は極楽の蓮池のふちに立って､この一部始終をじっと見ていらっしゃいましたが､やがてｶﾝﾀﾞﾀが血の池の底へ石のように沈んでしまいますと､悲しそうな御顔をなさりながら､またぶらぶら御歩きになり始めました｡自分ばかり地獄からぬけ出そうとする､ｶﾝﾀﾞﾀの無慈悲な心が､そうしてその心相当な罰をうけて､元の地獄へ落ちてしまったのが､御釈迦様の御目から見ると､浅間しく思召されたのでございましょう｡
 しかし極楽の蓮池の蓮は､少しもそんな事には頓着致しません｡その玉のような白い花は､御釈迦様の御足のまわりに､ゆらゆら萼を動かして､そのまん中にある金色の蕊からは､何とも云えない好い匂が､絶間なくあたりへ溢れて居ります｡極楽ももう午に近くなったのでございましょう｡"""

    },
    {
    "@search.action": "upload",
    "BookId": "128",
    "BookName": "羅生門",
    "BookAuthor": "芥川龍之介",
    "BookGenre": "短編小説",
    "Release_Date": "1915年11月",
    "Rating": "3.33",
    "Text": """ある日の暮方の事である｡一人の下人が､羅生門の下で雨やみを待っていた｡
 広い門の下には､この男のほかに誰もいない｡ただ､所々丹塗の剥げた､大きな円柱に､蟋蟀が一匹とまっている｡羅生門が､朱雀大路にある以上は､この男のほかにも､雨やみをする市女笠や揉烏帽子が､もう二三人はありそうなものである｡それが､この男のほかには誰もいない｡
 何故かと云うと､この二三年､京都には､地震とか辻風とか火事とか饑饉とか云う災がつづいて起った｡そこで洛中のさびれ方は一通りではない｡旧記によると､仏像や仏具を打砕いて､その丹がついたり､金銀の箔がついたりした木を､路ばたにつみ重ねて､薪の料に売っていたと云う事である｡洛中がその始末であるから､羅生門の修理などは､元より誰も捨てて顧る者がなかった｡するとその荒れ果てたのをよい事にして､狐狸が棲む｡盗人が棲む｡とうとうしまいには､引取り手のない死人を､この門へ持って来て､棄てて行くと云う習慣さえ出来た｡そこで､日の目が見えなくなると､誰でも気味を悪るがって､この門の近所へは足ぶみをしない事になってしまったのである｡
 その代りまた鴉がどこからか､たくさん集って来た｡昼間見ると､その鴉が何羽となく輪を描いて､高い鴟尾のまわりを啼きながら､飛びまわっている｡ことに門の上の空が､夕焼けであかくなる時には､それが胡麻をまいたようにはっきり見えた｡鴉は､勿論､門の上にある死人の肉を､啄みに来るのである｡――もっとも今日は､刻限が遅いせいか､一羽も見えない｡ただ､所々､崩れかかった､そうしてその崩れ目に長い草のはえた石段の上に､鴉の糞が､点々と白くこびりついているのが見える｡下人は七段ある石段の一番上の段に､洗いざらした紺の襖の尻を据えて､右の頬に出来た､大きな面皰を気にしながら､ぼんやり､雨のふるのを眺めていた｡
 作者はさっき､｢下人が雨やみを待っていた｣と書いた｡しかし､下人は雨がやんでも､格別どうしようと云う当てはない｡ふだんなら､勿論､主人の家へ帰る可き筈である｡所がその主人からは､四五日前に暇を出された｡前にも書いたように､当時京都の町は一通りならず衰微していた｡今この下人が､永年､使われていた主人から､暇を出されたのも､実はこの衰微の小さな余波にほかならない｡だから｢下人が雨やみを待っていた｣と云うよりも｢雨にふりこめられた下人が､行き所がなくて､途方にくれていた｣と云う方が､適当である｡その上､今日の空模様も少からず､この平安朝の下人の Sentimentalisme に影響した｡申の刻下りからふり出した雨は､いまだに上るけしきがない｡そこで､下人は､何をおいても差当り明日の暮しをどうにかしようとして――云わばどうにもならない事を､どうにかしようとして､とりとめもない考えをたどりながら､さっきから朱雀大路にふる雨の音を､聞くともなく聞いていたのである｡
 雨は､羅生門をつつんで､遠くから､ざあっと云う音をあつめて来る｡夕闇は次第に空を低くして､見上げると､門の屋根が､斜につき出した甍の先に､重たくうす暗い雲を支えている｡
 どうにもならない事を､どうにかするためには､手段を選んでいる遑はない｡選んでいれば､築土の下か､道ばたの土の上で､饑死をするばかりである｡そうして､この門の上へ持って来て､犬のように棄てられてしまうばかりである｡選ばないとすれば――下人の考えは､何度も同じ道を低徊した揚句に､やっとこの局所へ逢着した｡しかしこの｢すれば｣は､いつまでたっても､結局｢すれば｣であった｡下人は､手段を選ばないという事を肯定しながらも､この｢すれば｣のかたをつけるために､当然､その後に来る可き｢盗人になるよりほかに仕方がない｣と云う事を､積極的に肯定するだけの､勇気が出ずにいたのである｡
 下人は､大きな嚔をして､それから､大儀そうに立上った｡夕冷えのする京都は､もう火桶が欲しいほどの寒さである｡風は門の柱と柱との間を､夕闇と共に遠慮なく､吹きぬける｡丹塗の柱にとまっていた蟋蟀も､もうどこかへ行ってしまった｡
 下人は､頸をちぢめながら､山吹の汗袗に重ねた､紺の襖の肩を高くして門のまわりを見まわした｡雨風の患のない､人目にかかる惧のない､一晩楽にねられそうな所があれば､そこでともかくも､夜を明かそうと思ったからである｡すると､幸い門の上の楼へ上る､幅の広い､これも丹を塗った梯子が眼についた｡上なら､人がいたにしても､どうせ死人ばかりである｡下人はそこで､腰にさげた聖柄の太刀が鞘走らないように気をつけながら､藁草履をはいた足を､その梯子の一番下の段へふみかけた｡
 それから､何分かの後である｡羅生門の楼の上へ出る､幅の広い梯子の中段に､一人の男が､猫のように身をちぢめて､息を殺しながら､上の容子を窺っていた｡楼の上からさす火の光が､かすかに､その男の右の頬をぬらしている｡短い鬚の中に､赤く膿を持った面皰のある頬である｡下人は､始めから､この上にいる者は､死人ばかりだと高を括っていた｡それが､梯子を二三段上って見ると､上では誰か火をとぼして､しかもその火をそこここと動かしているらしい｡これは､その濁った､黄いろい光が､隅々に蜘蛛の巣をかけた天井裏に､揺れながら映ったので､すぐにそれと知れたのである｡この雨の夜に､この羅生門の上で､火をともしているからは､どうせただの者ではない｡
 下人は､守宮のように足音をぬすんで､やっと急な梯子を､一番上の段まで這うようにして上りつめた｡そうして体を出来るだけ､平にしながら､頸を出来るだけ､前へ出して､恐る恐る､楼の内を覗いて見た｡
 見ると､楼の内には､噂に聞いた通り､幾つかの死骸が､無造作に棄ててあるが､火の光の及ぶ範囲が､思ったより狭いので､数は幾つともわからない｡ただ､おぼろげながら､知れるのは､その中に裸の死骸と､着物を着た死骸とがあるという事である｡勿論､中には女も男もまじっているらしい｡そうして､その死骸は皆､それが､かつて､生きていた人間だと云う事実さえ疑われるほど､土を捏ねて造った人形のように､口を開いたり手を延ばしたりして､ごろごろ床の上にころがっていた｡しかも､肩とか胸とかの高くなっている部分に､ぼんやりした火の光をうけて､低くなっている部分の影を一層暗くしながら､永久に唖の如く黙っていた｡
 下人は､それらの死骸の腐爛した臭気に思わず､鼻を掩った｡しかし､その手は､次の瞬間には､もう鼻を掩う事を忘れていた｡ある強い感情が､ほとんどことごとくこの男の嗅覚を奪ってしまったからだ｡
 下人の眼は､その時､はじめてその死骸の中に蹲っている人間を見た｡檜皮色の着物を着た､背の低い､痩せた､白髪頭の､猿のような老婆である｡その老婆は､右の手に火をともした松の木片を持って､その死骸の一つの顔を覗きこむように眺めていた｡髪の毛の長い所を見ると､多分女の死骸であろう｡
 下人は､六分の恐怖と四分の好奇心とに動かされて､暫時は呼吸をするのさえ忘れていた｡旧記の記者の語を借りれば､｢頭身の毛も太る｣ように感じたのである｡すると老婆は､松の木片を､床板の間に挿して､それから､今まで眺めていた死骸の首に両手をかけると､丁度､猿の親が猿の子の虱をとるように､その長い髪の毛を一本ずつ抜きはじめた｡髪は手に従って抜けるらしい｡
 その髪の毛が､一本ずつ抜けるのに従って､下人の心からは､恐怖が少しずつ消えて行った｡そうして､それと同時に､この老婆に対するはげしい憎悪が､少しずつ動いて来た｡――いや､この老婆に対すると云っては､語弊があるかも知れない｡むしろ､あらゆる悪に対する反感が､一分毎に強さを増して来たのである｡この時､誰かがこの下人に､さっき門の下でこの男が考えていた､饑死をするか盗人になるかと云う問題を､改めて持出したら､恐らく下人は､何の未練もなく､饑死を選んだ事であろう｡それほど､この男の悪を憎む心は､老婆の床に挿した松の木片のように､勢いよく燃え上り出していたのである｡
 下人には､勿論､何故老婆が死人の髪の毛を抜くかわからなかった｡従って､合理的には､それを善悪のいずれに片づけてよいか知らなかった｡しかし下人にとっては､この雨の夜に､この羅生門の上で､死人の髪の毛を抜くと云う事が､それだけで既に許すべからざる悪であった｡勿論､下人は､さっきまで自分が､盗人になる気でいた事なぞは､とうに忘れていたのである｡
 そこで､下人は､両足に力を入れて､いきなり､梯子から上へ飛び上った｡そうして聖柄の太刀に手をかけながら､大股に老婆の前へ歩みよった｡老婆が驚いたのは云うまでもない｡
 老婆は､一目下人を見ると､まるで弩にでも弾かれたように､飛び上った｡
｢おのれ､どこへ行く｡｣
 下人は､老婆が死骸につまずきながら､慌てふためいて逃げようとする行手を塞いで､こう罵った｡老婆は､それでも下人をつきのけて行こうとする｡下人はまた､それを行かすまいとして､押しもどす｡二人は死骸の中で､しばらく､無言のまま､つかみ合った｡しかし勝敗は､はじめからわかっている｡下人はとうとう､老婆の腕をつかんで､無理にそこへﾈじ倒した｡丁度､鶏の脚のような､骨と皮ばかりの腕である｡
｢何をしていた｡云え｡云わぬと､これだぞよ｡｣
 下人は､老婆をつき放すと､いきなり､太刀の鞘を払って､白い鋼の色をその眼の前へつきつけた｡けれども､老婆は黙っている｡両手をわなわなふるわせて､肩で息を切りながら､眼を､眼球がﾏﾌﾞﾀの外へ出そうになるほど､見開いて､唖のように執拗く黙っている｡これを見ると､下人は始めて明白にこの老婆の生死が､全然､自分の意志に支配されていると云う事を意識した｡そうしてこの意識は､今までけわしく燃えていた憎悪の心を､いつの間にか冷ましてしまった｡後に残ったのは､ただ､ある仕事をして､それが円満に成就した時の､安らかな得意と満足とがあるばかりである｡そこで､下人は､老婆を見下しながら､少し声を柔らげてこう云った｡
｢己は検非違使の庁の役人などではない｡今し方この門の下を通りかかった旅の者だ｡だからお前に縄をかけて､どうしようと云うような事はない｡ただ､今時分この門の上で､何をして居たのだか､それを己に話しさえすればいいのだ｡｣
 すると､老婆は､見開いていた眼を､一層大きくして､じっとその下人の顔を見守った｡ﾏﾌﾞﾀの赤くなった､肉食鳥のような､鋭い眼で見たのである｡それから､皺で､ほとんど､鼻と一つになった唇を､何か物でも噛んでいるように動かした｡細い喉で､尖った喉仏の動いているのが見える｡その時､その喉から､鴉の啼くような声が､喘ぎ喘ぎ､下人の耳へ伝わって来た｡
｢この髪を抜いてな､この髪を抜いてな､鬘にしようと思うたのじゃ｡｣
 下人は､老婆の答が存外､平凡なのに失望した｡そうして失望すると同時に､また前の憎悪が､冷やかな侮蔑と一しょに､心の中へはいって来た｡すると､その気色が､先方へも通じたのであろう｡老婆は､片手に､まだ死骸の頭から奪った長い抜け毛を持ったなり､蟇のつぶやくような声で､口ごもりながら､こんな事を云った｡
｢成程な､死人の髪の毛を抜くと云う事は､何ぼう悪い事かも知れぬ｡じゃが､ここにいる死人どもは､皆､そのくらいな事を､されてもいい人間ばかりだぞよ｡現在､わしが今､髪を抜いた女などはな､蛇を四寸ばかりずつに切って干したのを､干魚だと云うて､太刀帯の陣へ売りに往んだわ｡疫病にかかって死ななんだら､今でも売りに往んでいた事であろ｡それもよ､この女の売る干魚は､味がよいと云うて､太刀帯どもが､欠かさず菜料に買っていたそうな｡わしは､この女のした事が悪いとは思うていぬ｡せねば､饑死をするのじゃて､仕方がなくした事であろ｡されば､今また､わしのしていた事も悪い事とは思わぬぞよ｡これとてもやはりせねば､饑死をするじゃて､仕方がなくする事じゃわいの｡じゃて､その仕方がない事を､よく知っていたこの女は､大方わしのする事も大目に見てくれるであろ｡｣
 老婆は､大体こんな意味の事を云った｡
 下人は､太刀を鞘におさめて､その太刀の柄を左の手でおさえながら､冷然として､この話を聞いていた｡勿論､右の手では､赤く頬に膿を持った大きな面皰を気にしながら､聞いているのである｡しかし､これを聞いている中に､下人の心には､ある勇気が生まれて来た｡それは､さっき門の下で､この男には欠けていた勇気である｡そうして､またさっきこの門の上へ上って､この老婆を捕えた時の勇気とは､全然､反対な方向に動こうとする勇気である｡下人は､饑死をするか盗人になるかに､迷わなかったばかりではない｡その時のこの男の心もちから云えば､饑死などと云う事は､ほとんど､考える事さえ出来ないほど､意識の外に追い出されていた｡
｢きっと､そうか｡｣
 老婆の話が完ると､下人は嘲るような声で念を押した｡そうして､一足前へ出ると､不意に右の手を面皰から離して､老婆の襟上をつかみながら､噛みつくようにこう云った｡
｢では､己が引剥をしようと恨むまいな｡己もそうしなければ､饑死をする体なのだ｡｣
 下人は､すばやく､老婆の着物を剥ぎとった｡それから､足にしがみつこうとする老婆を､手荒く死骸の上へ蹴倒した｡梯子の口までは､僅に五歩を数えるばかりである｡下人は､剥ぎとった檜皮色の着物をわきにかかえて､またたく間に急な梯子を夜の底へかけ下りた｡
 しばらく､死んだように倒れていた老婆が､死骸の中から､その裸の体を起したのは､それから間もなくの事である｡老婆はつぶやくような､うめくような声を立てながら､まだ燃えている火の光をたよりに､梯子の口まで､這って行った｡そうして､そこから､短い白髪を倒にして､門の下を覗きこんだ｡外には､ただ､黒洞々たる夜があるばかりである｡
 下人の行方は､誰も知らない｡"""
    },
    {
    "@search.action": "upload",
    "BookId": "42",
    "BookName": "鼻",
    "BookAuthor": "芥川龍之介",
    "BookGenre": "掌編小説",
    "Release_Date": "1916年2月",
    "Rating": "3.29",
    "Text": """禅智内供の鼻と云えば､池の尾で知らない者はない｡長さは五六寸あって上唇の上から顋の下まで下っている｡形は元も先も同じように太い｡云わば細長い腸詰めのような物が､ぶらりと顔のまん中からぶら下っているのである｡
 五十歳を越えた内供は､沙弥の昔から､内道場供奉の職に陞った今日まで､内心では始終この鼻を苦に病んで来た｡勿論表面では､今でもさほど気にならないような顔をしてすましている｡これは専念に当来の浄土を渇仰すべき僧侶の身で､鼻の心配をするのが悪いと思ったからばかりではない｡それよりむしろ､自分で鼻を気にしていると云う事を､人に知られるのが嫌だったからである｡内供は日常の談話の中に､鼻と云う語が出て来るのを何よりも惧れていた｡
 内供が鼻を持てあました理由は二つある｡――一つは実際的に､鼻の長いのが不便だったからである｡第一飯を食う時にも独りでは食えない｡独りで食えば､鼻の先が鋺の中の飯へとどいてしまう｡そこで内供は弟子の一人を膳の向うへ坐らせて､飯を食う間中､広さ一寸長さ二尺ばかりの板で､鼻を持上げていて貰う事にした｡しかしこうして飯を食うと云う事は､持上げている弟子にとっても､持上げられている内供にとっても､決して容易な事ではない｡一度この弟子の代りをした中童子が､嚏をした拍子に手がふるえて､鼻を粥の中へ落した話は､当時京都まで喧伝された｡――けれどもこれは内供にとって､決して鼻を苦に病んだ重な理由ではない｡内供は実にこの鼻によって傷つけられる自尊心のために苦しんだのである｡
 池の尾の町の者は､こう云う鼻をしている禅智内供のために､内供の俗でない事を仕合せだと云った｡あの鼻では誰も妻になる女があるまいと思ったからである｡中にはまた､あの鼻だから出家したのだろうと批評する者さえあった｡しかし内供は､自分が僧であるために､幾分でもこの鼻に煩される事が少くなったと思っていない｡内供の自尊心は､妻帯と云うような結果的な事実に左右されるためには､余りにﾃﾞﾘｹｲﾄに出来ていたのである｡そこで内供は､積極的にも消極的にも､この自尊心の毀損を恢復しようと試みた｡
 第一に内供の考えたのは､この長い鼻を実際以上に短く見せる方法である｡これは人のいない時に､鏡へ向って､いろいろな角度から顔を映しながら､熱心に工夫を凝らして見た｡どうかすると､顔の位置を換えるだけでは､安心が出来なくなって､頬杖をついたり頤の先へ指をあてがったりして､根気よく鏡を覗いて見る事もあった｡しかし自分でも満足するほど､鼻が短く見えた事は､これまでにただの一度もない｡時によると､苦心すればするほど､かえって長く見えるような気さえした｡内供は､こう云う時には､鏡を箱へしまいながら､今更のようにため息をついて､不承不承にまた元の経机へ､観音経をよみに帰るのである｡
 それからまた内供は､絶えず人の鼻を気にしていた｡池の尾の寺は､僧供講説などのしばしば行われる寺である｡寺の内には､僧坊が隙なく建て続いて､湯屋では寺の僧が日毎に湯を沸かしている｡従ってここへ出入する僧俗の類も甚だ多い｡内供はこう云う人々の顔を根気よく物色した｡一人でも自分のような鼻のある人間を見つけて､安心がしたかったからである｡だから内供の眼には､紺の水干も白の帷子もはいらない｡まして柑子色の帽子や､椎鈍の法衣なぞは､見慣れているだけに､有れども無きが如くである｡内供は人を見ずに､ただ､鼻を見た｡――しかし鍵鼻はあっても､内供のような鼻は一つも見当らない｡その見当らない事が度重なるに従って､内供の心は次第にまた不快になった｡内供が人と話しながら､思わずぶらりと下っている鼻の先をつまんで見て､年甲斐もなく顔を赤らめたのは､全くこの不快に動かされての所為である｡
 最後に､内供は､内典外典の中に､自分と同じような鼻のある人物を見出して､せめても幾分の心やりにしようとさえ思った事がある｡けれども､目連や､舎利弗の鼻が長かったとは､どの経文にも書いてない｡勿論竜樹や馬鳴も､人並の鼻を備えた菩薩である｡内供は､震旦の話の序に蜀漢の劉玄徳の耳が長かったと云う事を聞いた時に､それが鼻だったら､どのくらい自分は心細くなくなるだろうと思った｡
 内供がこう云う消極的な苦心をしながらも､一方ではまた､積極的に鼻の短くなる方法を試みた事は､わざわざここに云うまでもない｡内供はこの方面でもほとんど出来るだけの事をした｡烏瓜を煎じて飲んで見た事もある｡鼠の尿を鼻へなすって見た事もある｡しかし何をどうしても､鼻は依然として､五六寸の長さをぶらりと唇の上にぶら下げているではないか｡
 所がある年の秋､内供の用を兼ねて､京へ上った弟子の僧が､知己の医者から長い鼻を短くする法を教わって来た｡その医者と云うのは､もと震旦から渡って来た男で､当時は長楽寺の供僧になっていたのである｡
 内供は､いつものように､鼻などは気にかけないと云う風をして､わざとその法もすぐにやって見ようとは云わずにいた｡そうして一方では､気軽な口調で､食事の度毎に､弟子の手数をかけるのが､心苦しいと云うような事を云った｡内心では勿論弟子の僧が､自分を説伏せて､この法を試みさせるのを待っていたのである｡弟子の僧にも､内供のこの策略がわからない筈はない｡しかしそれに対する反感よりは､内供のそう云う策略をとる心もちの方が､より強くこの弟子の僧の同情を動かしたのであろう｡弟子の僧は､内供の予期通り､口を極めて､この法を試みる事を勧め出した｡そうして､内供自身もまた､その予期通り､結局この熱心な勧告に聴従する事になった｡
 その法と云うのは､ただ､湯で鼻を茹でて､その鼻を人に踏ませると云う､極めて簡単なものであった｡
 湯は寺の湯屋で､毎日沸かしている｡そこで弟子の僧は､指も入れられないような熱い湯を､すぐに提に入れて､湯屋から汲んで来た｡しかしじかにこの提へ鼻を入れるとなると､湯気に吹かれて顔を火傷する惧がある｡そこで折敷へ穴をあけて､それを提の蓋にして､その穴から鼻を湯の中へ入れる事にした｡鼻だけはこの熱い湯の中へ浸しても､少しも熱くないのである｡しばらくすると弟子の僧が云った｡
 ――もう茹った時分でござろう｡
 内供は苦笑した｡これだけ聞いたのでは､誰も鼻の話とは気がつかないだろうと思ったからである｡鼻は熱湯に蒸されて､蚤の食ったようにむず痒い｡
 弟子の僧は､内供が折敷の穴から鼻をぬくと､そのまだ湯気の立っている鼻を､両足に力を入れながら､踏みはじめた｡内供は横になって､鼻を床板の上へのばしながら､弟子の僧の足が上下に動くのを眼の前に見ているのである｡弟子の僧は､時々気の毒そうな顔をして､内供の禿げ頭を見下しながら､こんな事を云った｡
 ――痛うはござらぬかな｡医師は責めて踏めと申したで｡じゃが､痛うはござらぬかな｡
 内供は首を振って､痛くないと云う意味を示そうとした｡所が鼻を踏まれているので思うように首が動かない｡そこで､上眼を使って､弟子の僧の足に皹のきれているのを眺めながら､腹を立てたような声で､
――痛うはないて｡
 と答えた｡実際鼻はむず痒い所を踏まれるので､痛いよりもかえって気もちのいいくらいだったのである｡
 しばらく踏んでいると､やがて､粟粒のようなものが､鼻へ出来はじめた｡云わば毛をむしった小鳥をそっくり丸炙にしたような形である｡弟子の僧はこれを見ると､足を止めて独り言のようにこう云った｡
 ――これを鑷子でぬけと申す事でござった｡
 内供は､不足らしく頬をふくらせて､黙って弟子の僧のするなりに任せて置いた｡勿論弟子の僧の親切がわからない訳ではない｡それは分っても､自分の鼻をまるで物品のように取扱うのが､不愉快に思われたからである｡内供は､信用しない医者の手術をうける患者のような顔をして､不承不承に弟子の僧が､鼻の毛穴から鑷子で脂をとるのを眺めていた｡脂は､鳥の羽の茎のような形をして､四分ばかりの長さにぬけるのである｡
 やがてこれが一通りすむと､弟子の僧は､ほっと一息ついたような顔をして､
 ――もう一度､これを茹でればようござる｡
 と云った｡
 内供はやはり､八の字をよせたまま不服らしい顔をして､弟子の僧の云うなりになっていた｡
 さて二度目に茹でた鼻を出して見ると､成程､いつになく短くなっている｡これではあたりまえの鍵鼻と大した変りはない｡内供はその短くなった鼻を撫でながら､弟子の僧の出してくれる鏡を､極りが悪るそうにおずおず覗いて見た｡
 鼻は――あの顋の下まで下っていた鼻は､ほとんど嘘のように萎縮して､今は僅に上唇の上で意気地なく残喘を保っている｡所々まだらに赤くなっているのは､恐らく踏まれた時の痕であろう｡こうなれば､もう誰も哂うものはないにちがいない｡――鏡の中にある内供の顔は､鏡の外にある内供の顔を見て､満足そうに眼をしばたたいた｡
 しかし､その日はまだ一日､鼻がまた長くなりはしないかと云う不安があった｡そこで内供は誦経する時にも､食事をする時にも､暇さえあれば手を出して､そっと鼻の先にさわって見た｡が､鼻は行儀よく唇の上に納まっているだけで､格別それより下へぶら下って来る景色もない｡それから一晩寝てあくる日早く眼がさめると内供はまず､第一に､自分の鼻を撫でて見た｡鼻は依然として短い｡内供はそこで､幾年にもなく､法華経書写の功を積んだ時のような､のびのびした気分になった｡
 所が二三日たつ中に､内供は意外な事実を発見した｡それは折から､用事があって､池の尾の寺を訪れた侍が､前よりも一層可笑しそうな顔をして､話も碌々せずに､じろじろ内供の鼻ばかり眺めていた事である｡それのみならず､かつて､内供の鼻を粥の中へ落した事のある中童子なぞは､講堂の外で内供と行きちがった時に､始めは､下を向いて可笑しさをこらえていたが､とうとうこらえ兼ねたと見えて､一度にふっと吹き出してしまった｡用を云いつかった下法師たちが､面と向っている間だけは､慎んで聞いていても､内供が後さえ向けば､すぐにくすくす笑い出したのは､一度や二度の事ではない｡
 内供ははじめ､これを自分の顔がわりがしたせいだと解釈した｡しかしどうもこの解釈だけでは十分に説明がつかないようである｡――勿論､中童子や下法師が哂う原因は､そこにあるのにちがいない｡けれども同じ哂うにしても､鼻の長かった昔とは､哂うのにどことなく容子がちがう｡見慣れた長い鼻より､見慣れない短い鼻の方が滑稽に見えると云えば､それまでである｡が､そこにはまだ何かあるらしい｡
 ――前にはあのようにつけつけとは哂わなんだて｡
 内供は､誦しかけた経文をやめて､禿げ頭を傾けながら､時々こう呟く事があった｡愛すべき内供は､そう云う時になると､必ずぼんやり､傍にかけた普賢の画像を眺めながら､鼻の長かった四五日前の事を憶い出して､｢今はむげにいやしくなりさがれる人の､さかえたる昔をしのぶがごとく｣ふさぎこんでしまうのである｡――内供には､遺憾ながらこの問に答を与える明が欠けていた｡
 ――人間の心には互に矛盾した二つの感情がある｡勿論､誰でも他人の不幸に同情しない者はない｡所がその人がその不幸を､どうにかして切りぬける事が出来ると､今度はこっちで何となく物足りないような心もちがする｡少し誇張して云えば､もう一度その人を､同じ不幸に陥れて見たいような気にさえなる｡そうしていつの間にか､消極的ではあるが､ある敵意をその人に対して抱くような事になる｡――内供が､理由を知らないながらも､何となく不快に思ったのは､池の尾の僧俗の態度に､この傍観者の利己主義をそれとなく感づいたからにほかならない｡
 そこで内供は日毎に機嫌が悪くなった｡二言目には､誰でも意地悪く叱りつける｡しまいには鼻の療治をしたあの弟子の僧でさえ､｢内供は法慳貪の罪を受けられるぞ｣と陰口をきくほどになった｡殊に内供を怒らせたのは､例の悪戯な中童子である｡ある日､けたたましく犬の吠える声がするので､内供が何気なく外へ出て見ると､中童子は､二尺ばかりの木の片をふりまわして､毛の長い､痩せた尨犬を逐いまわしている｡それもただ､逐いまわしているのではない｡｢鼻を打たれまい｡それ､鼻を打たれまい｣と囃しながら､逐いまわしているのである｡内供は､中童子の手からその木の片をひったくって､したたかその顔を打った｡木の片は以前の鼻持上げの木だったのである｡
 内供はなまじいに､鼻の短くなったのが､かえって恨めしくなった｡
 するとある夜の事である｡日が暮れてから急に風が出たと見えて､塔の風鐸の鳴る音が､うるさいほど枕に通って来た｡その上､寒さもめっきり加わったので､老年の内供は寝つこうとしても寝つかれない｡そこで床の中でまじまじしていると､ふと鼻がいつになく､むず痒いのに気がついた｡手をあてて見ると少し水気が来たようにむくんでいる｡どうやらそこだけ､熱さえもあるらしい｡
 ――無理に短うしたで､病が起ったのかも知れぬ｡
 内供は､仏前に香花を供えるような恭しい手つきで､鼻を抑えながら､こう呟いた｡
 翌朝､内供がいつものように早く眼をさまして見ると､寺内の銀杏や橡が一晩の中に葉を落したので､庭は黄金を敷いたように明るい｡塔の屋根には霜が下りているせいであろう｡まだうすい朝日に､九輪がまばゆく光っている｡禅智内供は､蔀を上げた縁に立って､深く息をすいこんだ｡
 ほとんど､忘れようとしていたある感覚が､再び内供に帰って来たのはこの時である｡
 内供は慌てて鼻へ手をやった｡手にさわるものは､昨夜の短い鼻ではない｡上唇の上から顋の下まで､五六寸あまりもぶら下っている､昔の長い鼻である｡内供は鼻が一夜の中に､また元の通り長くなったのを知った｡そうしてそれと同時に､鼻が短くなった時と同じような､はればれした心もちが､どこからともなく帰って来るのを感じた｡
 ――こうなれば､もう誰も哂うものはないにちがいない｡
 内供は心の中でこう自分に囁いた｡長い鼻をあけ方の秋風にぶらつかせながら｡"""
    },
    {
    "@search.action": "upload",
    "BookId": "69",
    "BookName": "河童",
    "BookAuthor": "芥川龍之介",
    "BookGenre": "短編小説",
    "Release_Date": "1927年3月",
    "Rating": "3.57",
    "Text": """序
 これはある精神病院の患者､――第二十三号がだれにでもしゃべる話である｡彼はもう三十を越しているであろう｡が､一見したところはいかにも若々しい狂人である｡彼の半生の経験は､――いや､そんなことはどうでもよい｡彼はただじっと両膝をかかえ､時々窓の外へ目をやりながら､(鉄格子をはめた窓の外には枯れ葉さえ見えない樫の木が一本､雪曇りの空に枝を張っていた｡)院長のS博士や僕を相手に長々とこの話をしゃべりつづけた｡もっとも身ぶりはしなかったわけではない｡彼はたとえば｢驚いた｣と言う時には急に顔をのけぞらせたりした｡……
 僕はこういう彼の話をかなり正確に写したつもりである｡もしまただれか僕の筆記に飽き足りない人があるとすれば､東京市外村のS精神病院を尋ねてみるがよい｡年よりも若い第二十三号はまず丁寧に頭を下げ､蒲団のない椅子を指さすであろう｡それから憂鬱な微笑を浮かべ､静かにこの話を繰り返すであろう｡最後に､――僕はこの話を終わった時の彼の顔色を覚えている｡彼は最後に身を起こすが早いか､たちまち拳骨をふりまわしながら､だれにでもこう怒鳴りつけるであろう｡――｢出て行け! この悪党めが! 貴様も莫迦な､嫉妬深い､猥褻な､ずうずうしい､うぬぼれきった､残酷な､虫のいい動物なんだろう｡出ていけ! この悪党めが!｣
一
 三年前の夏のことです｡僕は人並みにﾘｭｯｸ･ｻｯｸを背負い､あの上高地の温泉宿から穂高山へ登ろうとしました｡穂高山へ登るのには御承知のとおり梓川をさかのぼるほかはありません｡僕は前に穂高山はもちろん､槍ヶ岳にも登っていましたから､朝霧の下りた梓川の谷を案内者もつれずに登ってゆきました｡朝霧の下りた梓川の谷を――しかしその霧はいつまでたっても晴れる景色は見えません｡のみならずかえって深くなるのです｡僕は一時間ばかり歩いた後､一度は上高地の温泉宿へ引き返すことにしようかと思いました｡けれども上高地へ引き返すにしても､とにかく霧の晴れるのを待った上にしなければなりません｡といって霧は一刻ごとにずんずん深くなるばかりなのです｡｢ええ､いっそ登ってしまえ｡｣――僕はこう考えましたから､梓川の谷を離れないように熊笹の中を分けてゆきました｡
 しかし僕の目をさえぎるものはやはり深い霧ばかりです｡もっとも時々霧の中から太い毛生欅や樅の枝が青あおと葉を垂らしたのも見えなかったわけではありません｡それからまた放牧の馬や牛も突然僕の前へ顔を出しました｡けれどもそれらは見えたと思うと､たちまち濛々とした霧の中に隠れてしまうのです｡そのうちに足もくたびれてくれば､腹もだんだん減りはじめる､――おまけに霧にぬれ透った登山服や毛布なども並みたいていの重さではありません｡僕はとうとう我を折りましたから､岩にせかれている水の音をたよりに梓川の谷へ下りることにしました｡
 僕は水ぎわの岩に腰かけ､とりあえず食事にとりかかりました｡ｺｵﾝﾄﾞ･ﾋﾞｲﾌの罐を切ったり､枯れ枝を集めて火をつけたり､――そんなことをしているうちにかれこれ十分はたったでしょう｡その間にどこまでも意地の悪い霧はいつかほのぼのと晴れかかりました｡僕はﾊﾟﾝをかじりながら､ちょっと腕時計をのぞいてみました｡時刻はもう一時二十分過ぎです｡が､それよりも驚いたのは何か気味の悪い顔が一つ､円い腕時計の硝子の上へちらりと影を落としたことです｡僕は驚いてふり返りました｡すると､――僕が河童というものを見たのは実にこの時がはじめてだったのです｡僕の後ろにある岩の上には画にあるとおりの河童が一匹､片手は白樺の幹を抱え､片手は目の上にかざしたなり､珍しそうに僕を見おろしていました｡
 僕は呆っ気にとられたまま､しばらくは身動きもしずにいました｡河童もやはり驚いたとみえ､目の上の手さえ動かしません｡そのうちに僕は飛び立つが早いか､岩の上の河童へおどりかかりました｡同時にまた河童も逃げ出しました｡いや､おそらくは逃げ出したのでしょう｡実はひらりと身をかわしたと思うと､たちまちどこかへ消えてしまったのです｡僕はいよいよ驚きながら､熊笹の中を見まわしました｡すると河童は逃げ腰をしたなり､二三ﾒｴﾄﾙ隔たった向こうに僕を振り返って見ているのです｡それは不思議でもなんでもありません｡しかし僕に意外だったのは河童の体の色のことです｡岩の上に僕を見ていた河童は一面に灰色を帯びていました｡けれども今は体中すっかり緑いろに変わっているのです｡僕は｢畜生!｣とおお声をあげ､もう一度河童へ飛びかかりました｡河童が逃げ出したのはもちろんです｡それから僕は三十分ばかり､熊笹を突きぬけ､岩を飛び越え､遮二無二河童を追いつづけました｡
 河童もまた足の早いことは決して猿などに劣りません｡僕は夢中になって追いかける間に何度もその姿を見失おうとしました｡のみならず足をすべらして転がったこともたびたびです｡が､大きい橡の木が一本､太ぶとと枝を張った下へ来ると､幸いにも放牧の牛が一匹､河童の往く先へ立ちふさがりました｡しかもそれは角の太い､目を血走らせた牡牛なのです｡河童はこの牡牛を見ると､何か悲鳴をあげながら､ひときわ高い熊笹の中へもんどりを打つように飛び込みました｡僕は､――僕も｢しめた｣と思いましたから､いきなりそのあとへ追いすがりました｡するとそこには僕の知らない穴でもあいていたのでしょう｡僕は滑らかな河童の背中にやっと指先がさわったと思うと､たちまち深い闇の中へまっさかさまに転げ落ちました｡が､我々人間の心はこういう危機一髪の際にも途方もないことを考えるものです｡僕は｢あっ｣と思う拍子にあの上高地の温泉宿のそばに｢河童橋｣という橋があるのを思い出しました｡それから､――それから先のことは覚えていません｡僕はただ目の前に稲妻に似たものを感じたぎり､いつの間にか正気を失っていました｡
二
 そのうちにやっと気がついてみると､僕は仰向けに倒れたまま､大勢の河童にとり囲まれていました｡のみならず太い嘴の上に鼻目金をかけた河童が一匹､僕のそばへひざまずきながら､僕の胸へ聴診器を当てていました｡その河童は僕が目をあいたのを見ると､僕に｢静かに｣という手真似をし､それからだれか後ろにいる河童へ Quax, quax と声をかけました｡するとどこからか河童が二匹､担架を持って歩いてきました｡僕はこの担架にのせられたまま､大勢の河童の群がった中を静かに何町か進んでゆきました｡僕の両側に並んでいる町は少しも銀座通りと違いありません｡やはり毛生欅の並み木のかげにいろいろの店が日除けを並べ､そのまた並み木にはさまれた道を自動車が何台も走っているのです｡
 やがて僕を載せた担架は細い横町を曲ったと思うと､ある家の中へかつぎこまれました｡それは後に知ったところによれば､あの鼻目金をかけた河童の家､――ﾁｬｯｸという医者の家だったのです｡ﾁｬｯｸは僕を小ぎれいなﾍﾞｯﾄﾞの上へ寝かせました｡それから何か透明な水薬を一杯飲ませました｡僕はﾍﾞｯﾄﾞの上に横たわったなり､ﾁｬｯｸのするままになっていました｡実際また僕の体はろくに身動きもできないほど､節々が痛んでいたのですから｡
 ﾁｬｯｸは一日に二三度は必ず僕を診察にきました｡また三日に一度ぐらいは僕の最初に見かけた河童､――ﾊﾞｯｸﾞという漁夫も尋ねてきました｡河童は我々人間が河童のことを知っているよりもはるかに人間のことを知っています｡それは我々人間が河童を捕獲することよりもずっと河童が人間を捕獲することが多いためでしょう｡捕獲というのは当たらないまでも､我々人間は僕の前にもたびたび河童の国へ来ているのです｡のみならず一生河童の国に住んでいたものも多かったのです｡なぜと言ってごらんなさい｡僕らはただ河童ではない､人間であるという特権のために働かずに食っていられるのです｡現にﾊﾞｯｸﾞの話によれば､ある若い道路工夫などはやはり偶然この国へ来た後､雌の河童を妻にめとり､死ぬまで住んでいたということです｡もっともそのまた雌の河童はこの国第一の美人だった上､夫の道路工夫をごまかすのにも妙をきわめていたということです｡
 僕は一週間ばかりたった後､この国の法律の定めるところにより､｢特別保護住民｣としてﾁｬｯｸの隣に住むことになりました｡僕の家は小さい割にいかにも瀟洒とできあがっていました｡もちろんこの国の文明は我々人間の国の文明――少なくとも日本の文明などとあまり大差はありません｡往来に面した客間の隅には小さいﾋﾟｱﾉが一台あり､それからまた壁には額縁へ入れたｴｯﾃｨﾝｸﾞなども懸っていました｡ただ肝腎の家をはじめ､ﾃｴﾌﾞﾙや椅子の寸法も河童の身長に合わせてありますから､子どもの部屋に入れられたようにそれだけは不便に思いました｡
 僕はいつも日暮れがたになると､この部屋にﾁｬｯｸやﾊﾞｯｸﾞを迎え､河童の言葉を習いました｡いや､彼らばかりではありません｡特別保護住民だった僕にだれも皆好奇心を持っていましたから､毎日血圧を調べてもらいに､わざわざﾁｬｯｸを呼び寄せるｹﾞｴﾙという硝子会社の社長などもやはりこの部屋へ顔を出したものです｡しかし最初の半月ほどの間に一番僕と親しくしたのはやはりあのﾊﾞｯｸﾞという漁夫だったのです｡
 ある生暖かい日の暮れです｡僕はこの部屋のﾃｴﾌﾞﾙを中に漁夫のﾊﾞｯｸﾞと向かい合っていました｡するとﾊﾞｯｸﾞはどう思ったか､急に黙ってしまった上､大きい目をいっそう大きくしてじっと僕を見つめました｡僕はもちろん妙に思いましたから､｢Quax, Bag, quo quel, quan?｣と言いました｡これは日本語に翻訳すれば､｢おい､ﾊﾞｯｸﾞ､どうしたんだ｣ということです｡が､ﾊﾞｯｸﾞは返事をしません｡のみならずいきなり立ち上がると､べろりと舌を出したなり､ちょうど蛙の跳ねるように飛びかかる気色さえ示しました｡僕はいよいよ無気味になり､そっと椅子から立ち上がると､一足飛びに戸口へ飛び出そうとしました｡ちょうどそこへ顔を出したのは幸いにも医者のﾁｬｯｸです｡
｢こら､ﾊﾞｯｸﾞ､何をしているのだ?｣
 ﾁｬｯｸは鼻目金をかけたまま､こういうﾊﾞｯｸﾞ[#｢ﾊﾞｯｸﾞ｣は底本では｢ﾊﾞｯｸ｣]をにらみつけました｡するとﾊﾞｯｸﾞは恐れいったとみえ､何度も頭へ手をやりながら､こう言ってﾁｬｯｸにあやまるのです｡
｢どうもまことに相すみません｡実はこの旦那の気味悪がるのがおもしろかったものですから､つい調子に乗って悪戯をしたのです｡どうか旦那も堪忍してください｡｣
三
 僕はこの先を話す前にちょっと河童というものを説明しておかなければなりません｡河童はいまだに実在するかどうかも疑問になっている動物です｡が､それは僕自身が彼らの間に住んでいた以上､少しも疑う余地はないはずです｡ではまたどういう動物かと言えば､頭に短い毛のあるのはもちろん､手足に水掻きのついていることも｢水虎考略｣などに出ているのと著しい違いはありません｡身長もざっと一ﾒｴﾄﾙを越えるか越えぬくらいでしょう｡体重は医者のﾁｬｯｸによれば､二十ﾎﾟﾝﾄﾞから三十ﾎﾟﾝﾄﾞまで､――まれには五十何ﾎﾟﾝﾄﾞぐらいの大河童もいると言っていました｡それから頭のまん中には楕円形の皿があり､そのまた皿は年齢により､だんだん固さを加えるようです｡現に年をとったﾊﾞｯｸﾞの皿は若いﾁｬｯｸの皿などとは全然手ざわりも違うのです｡しかし一番不思議なのは河童の皮膚の色のことでしょう｡河童は我々人間のように一定の皮膚の色を持っていません｡なんでもその周囲の色と同じ色に変わってしまう､――たとえば草の中にいる時には草のように緑色に変わり､岩の上にいる時には岩のように灰色に変わるのです｡これはもちろん河童に限らず､ｶﾒﾚｵﾝにもあることです｡あるいは河童は皮膚組織の上に何かｶﾒﾚｵﾝに近いところを持っているのかもしれません｡僕はこの事実を発見した時､西国の河童は緑色であり､東北の河童は赤いという民俗学上の記録を思い出しました｡のみならずﾊﾞｯｸﾞを追いかける時､突然どこへ行ったのか､見えなくなったことを思い出しました｡しかも河童は皮膚の下によほど厚い脂肪を持っているとみえ､この地下の国の温度は比較的低いのにもかかわらず､(平均華氏五十度前後です｡)着物というものを知らず[#｢知らず｣は底本では｢知らす｣]にいるのです｡もちろんどの河童も目金をかけたり､巻煙草の箱を携えたり､金入れを持ったりはしているでしょう｡しかし河童はｶﾝｶﾞﾙｳのように腹に袋を持っていますから､それらのものをしまう時にも格別不便はしないのです｡ただ僕におかしかったのは腰のまわりさえおおわないことです｡僕はある時この習慣をなぜかとﾊﾞｯｸﾞに尋ねてみました｡すると[#｢すると｣は底本では｢ずると｣]ﾊﾞｯｸﾞはのけぞったまま､いつまでもげらげら笑っていました｡おまけに｢わたしはお前さんの隠しているのがおかしい｣と返事をしました｡
四
 僕はだんだん河童の使う日常の言葉を覚えてきました｡従って河童の風俗や習慣ものみこめるようになってきました｡その中でも一番不思議だったのは河童は我々人間の真面目に思うことをおかしがる､同時に我々人間のおかしがることを真面目に思う――こういうとんちんかんな習慣です｡たとえば我々人間は正義とか人道とかいうことを真面目に思う､しかし河童はそんなことを聞くと､腹をかかえて笑い出すのです｡つまり彼らの滑稽という観念は我々の滑稽という観念と全然標準を異にしているのでしょう｡僕はある時医者のﾁｬｯｸと産児制限の話をしていました｡するとﾁｬｯｸは大口をあいて､鼻目金の落ちるほど笑い出しました｡僕はもちろん腹が立ちましたから､何がおかしいかと詰問しました｡なんでもﾁｬｯｸの返答はだいたいこうだったように覚えています｡もっとも多少細かいところは間違っているかもしれません｡なにしろまだそのころは僕も河童の使う言葉をすっかり理解していなかったのですから｡
｢しかし両親のつごうばかり考えているのはおかしいですからね｡どうもあまり手前勝手ですからね｡｣
 その代わりに我々人間から見れば､実際また河童のお産ぐらい､おかしいものはありません｡現に僕はしばらくたってから､ﾊﾞｯｸﾞの細君のお産をするところをﾊﾞｯｸﾞの小屋へ見物にゆきました｡河童もお産をする時には我々人間と同じことです｡やはり医者や産婆などの助けを借りてお産をするのです｡けれどもお産をするとなると､父親は電話でもかけるように母親の生殖器に口をつけ､｢お前はこの世界へ生まれてくるかどうか､よく考えた上で返事をしろ｡｣と大きな声で尋ねるのです｡ﾊﾞｯｸﾞもやはり膝をつきながら､何度も繰り返してこう言いました｡それからﾃｴﾌﾞﾙの上にあった消毒用の水薬でうがいをしました｡すると細君の腹の中の子は多少気兼ねでもしているとみえ､こう小声に返事をしました｡
｢僕は生まれたくはありません｡第一僕のお父さんの遺伝は精神病だけでもたいへんです｡その上僕は河童的存在を悪いと信じていますから｡｣
 ﾊﾞｯｸﾞはこの返事を聞いた時､てれたように頭をかいていました｡が､そこにい合わせた産婆はたちまち細君の生殖器へ太い硝子の管を突きこみ､何か液体を注射しました｡すると細君はほっとしたように太い息をもらしました｡同時にまた今まで大きかった腹は水素瓦斯を抜いた風船のようにへたへたと縮んでしまいました｡
 こういう返事をするくらいですから､河童の子どもは生まれるが早いか､もちろん歩いたりしゃべったりするのです｡なんでもﾁｬｯｸの話では出産後二十六日目に神の有無について講演をした子どももあったとかいうことです｡もっともその子どもは二月目には死んでしまったということですが｡
 お産の話をしたついでですから､僕がこの国へ来た三月目に偶然ある街の角で見かけた､大きいﾎﾟｽﾀｱの話をしましょう｡その大きいﾎﾟｽﾀｱの下には喇叭を吹いている河童だの剣を持っている河童だのが十二三匹描いてありました｡それからまた上には河童の使う､ちょうど時計のｾﾞﾝﾏｲに似た螺旋文字が一面に並べてありました｡この螺旋文字を翻訳すると､だいたいこういう意味になるのです｡これもあるいは細かいところは間違っているかもしれません｡が､とにかく僕としては僕といっしょに歩いていた､ﾗｯﾌﾟという河童の学生が大声に読み上げてくれる言葉をいちいちﾉｵﾄにとっておいたのです｡
		遺伝的義勇隊を募る
		健全なる男女の河童よ
		悪遺伝を撲滅するために
		不健全なる男女の河童と結婚せよ
 僕はもちろんその時にもそんなことの行なわれないことをﾗｯﾌﾟに話して聞かせました｡するとﾗｯﾌﾟばかりではない､ﾎﾟｽﾀｱの近所にいた河童はことごとくげらげら笑い出しました｡
｢行なわれない? だってあなたの話ではあなたがたもやはり我々のように行なっていると思いますがね｡あなたは令息が女中に惚れたり､令嬢が運転手に惚れたりするのはなんのためだと思っているのです? あれは皆無意識的に悪遺伝を撲滅しているのですよ｡第一この間あなたの話したあなたがた人間の義勇隊よりも､――一本の鉄道を奪うために互いに殺し合う義勇隊ですね､――ああいう義勇隊に比べれば､ずっと僕たちの義勇隊は高尚ではないかと思いますがね｡｣
 ﾗｯﾌﾟは真面目にこう言いながら､しかも太い腹だけはおかしそうに絶えず浪立たせていました｡が､僕は笑うどころか､あわててある河童をつかまえようとしました｡それは僕の油断を見すまし､その河童が僕の万年筆を盗んだことに気がついたからです｡しかし皮膚の滑らかな河童は容易に我々にはつかまりません｡その河童もぬらりとすべり抜けるが早いかいっさんに逃げ出してしまいました｡ちょうど蚊のようにやせた体を倒れるかと思うくらいのめらせながら｡
五
 僕はこのﾗｯﾌﾟという河童にﾊﾞｯｸﾞにも劣らぬ世話になりました｡が､その中でも忘れられないのはﾄｯｸという河童に紹介されたことです｡ﾄｯｸは河童仲間の詩人です｡詩人が髪を長くしていることは我々人間と変わりません｡僕は時々ﾄｯｸの家へ退屈しのぎに遊びにゆきました｡ﾄｯｸはいつも狭い部屋に高山植物の鉢植えを並べ､詩を書いたり煙草をのんだり､いかにも気楽そうに暮らしていました｡そのまた部屋の隅には雌の河童が一匹､(ﾄｯｸは自由恋愛家ですから､細君というものは持たないのです｡)編み物か何かしていました｡ﾄｯｸは僕の顔を見ると､いつも微笑してこう言うのです｡(もっとも河童の微笑するのはあまりいいものではありません｡少なくとも僕は最初のうちはむしろ無気味に感じたものです｡)
｢やあ､よく来たね｡まあ､その椅子にかけたまえ｡｣
 ﾄｯｸはよく河童の生活だの河童の芸術だのの話をしました｡ﾄｯｸの信ずるところによれば､当たり前の河童の生活ぐらい､莫迦げているものはありません｡親子夫婦兄弟などというのはことごとく互いに苦しめ合うことを唯一の楽しみにして暮らしているのです｡ことに家族制度というものは莫迦げている以上にも莫迦げているのです｡ﾄｯｸはある時窓の外を指さし､｢見たまえ｡あの莫迦げさ加減を!｣と吐き出すように言いました｡窓の外の往来にはまだ年の若い河童が一匹､両親らしい河童をはじめ､七八匹の雌雄の河童を頸のまわりへぶら下げながら､息も絶え絶えに歩いていました｡しかし僕は年の若い河童の犠牲的精神に感心しましたから､かえってその健気さをほめ立てました｡
｢ふん､君はこの国でも市民になる資格を持っている｡……時に君は社会主義者かね?｣
 僕はもちろん qua(これは河童の使う言葉では｢然り｣という意味を現わすのです｡)と答えました｡
｢では百人の凡人のために甘んじてひとりの天才を犠牲にすることも顧みないはずだ｡｣
｢では君は何主義者だ? だれかﾄｯｸ君の信条は無政府主義だと言っていたが､……｣
｢僕か? 僕は超人(直訳すれば超河童です｡)だ｡｣
 ﾄｯｸは昂然と言い放ちました｡こういうﾄｯｸは芸術の上にも独特な考えを持っています｡ﾄｯｸの信ずるところによれば､芸術は何ものの支配をも受けない､芸術のための芸術である､従って芸術家たるものは何よりも先に善悪を絶した超人でなければならぬというのです｡もっともこれは必ずしもﾄｯｸ一匹の意見ではありません｡ﾄｯｸの仲間の詩人たちはたいてい同意見を持っているようです｡現に僕はﾄｯｸといっしょにたびたび超人倶楽部へ遊びにゆきました｡超人倶楽部に集まってくるのは詩人､小説家､戯曲家､批評家､画家､音楽家､彫刻家､芸術上の素人等です｡しかしいずれも超人です｡彼らは電燈の明るいｻﾛﾝにいつも快活に話し合っていました｡のみならず時には得々と彼らの超人ぶりを示し合っていました｡たとえばある彫刻家などは大きい鬼羊歯の鉢植えの間に年の若い河童をつかまえながら､しきりに男色をもてあそんでいました｡またある雌の小説家などはﾃｴﾌﾞﾙの上に立ち上がったなり､ｱﾌﾞｻﾝﾄを六十本飲んで見せました｡もっともこれは六十本目にﾃｴﾌﾞﾙの下へ転げ落ちるが早いか､たちまち往生してしまいましたが｡
 僕はある月のいい晩､詩人のﾄｯｸと肘を組んだまま､超人倶楽部から帰ってきました｡ﾄｯｸはいつになく沈みこんでひとことも口をきかずにいました｡そのうちに僕らは火かげのさした､小さい窓の前を通りかかりました｡そのまた窓の向こうには夫婦らしい雌雄の河童が二匹､三匹の子どもの河童といっしょに晩餐のﾃｴﾌﾞﾙに向かっているのです｡するとﾄｯｸはため息をしながら､突然こう僕に話しかけました｡
｢僕は超人的恋愛家だと思っているがね､ああいう家庭の容子を見ると､やはりうらやましさを感じるんだよ｡｣
｢しかしそれはどう考えても､矛盾しているとは思わないかね?｣
 けれどもﾄｯｸは月明りの下にじっと腕を組んだまま､あの小さい窓の向こうを､――平和な五匹の河童たちの晩餐のﾃｴﾌﾞﾙを見守っていました｡それからしばらくしてこう答えました｡
｢あすこにある玉子焼きはなんと言っても､恋愛などよりも衛生的だからね｡｣
六
 実際また河童の恋愛は我々人間の恋愛とはよほど趣を異にしています｡雌の河童はこれぞという雄の河童を見つけるが早いか､雄の河童をとらえるのにいかなる手段も顧みません､一番正直な雌の河童は遮二無二雄の河童を追いかけるのです｡現に僕は気違いのように雄の河童を追いかけている雌の河童を見かけました｡いや､そればかりではありません｡若い雌の河童はもちろん､その河童の両親や兄弟までいっしょになって追いかけるのです｡雄の河童こそみじめです｡なにしろさんざん逃げまわったあげく､運よくつかまらずにすんだとしても､二三か月は床についてしまうのですから｡僕はある時僕の家にﾄｯｸの詩集を読んでいました｡するとそこへ駆けこんできたのはあのﾗｯﾌﾟという学生です｡ﾗｯﾌﾟは僕の家へ転げこむと､床の上へ倒れたなり､息も切れ切れにこう言うのです｡
｢大変だ! とうとう僕は抱きつかれてしまった!｣
 僕はとっさに詩集を投げ出し､戸口の錠をおろしてしまいました｡しかし鍵穴からのぞいてみると､硫黄の粉末を顔に塗った､背の低い雌の河童が一匹､まだ戸口にうろついているのです｡ﾗｯﾌﾟはその日から何週間か僕の床の上に寝ていました｡のみならずいつかﾗｯﾌﾟの嘴はすっかり腐って落ちてしまいました｡
 もっともまた時には雌の河童を一生懸命に追いかける雄の河童もないではありません｡しかしそれもほんとうのところは追いかけずにはいられないように雌の河童が仕向けるのです｡僕はやはり気違いのように雌の河童を追いかけている雄の河童も見かけました｡雌の河童は逃げてゆくうちにも､時々わざと立ち止まってみたり､四つん這いになったりして見せるのです｡おまけにちょうどいい時分になると､さもがっかりしたように楽々とつかませてしまうのです｡僕の見かけた雄の河童は雌の河童を抱いたなり､しばらくそこに転がっていました｡が､やっと起き上がったのを見ると､失望というか､後悔というか､とにかくなんとも形容できない､気の毒な顔をしていました｡しかしそれはまだいいのです｡これも僕の見かけた中に小さい雄の河童が一匹､雌の河童を追いかけていました｡雌の河童は例のとおり､誘惑的遁走をしているのです｡するとそこへ向こうの街から大きい雄の河童が一匹､鼻息を鳴らせて歩いてきました｡雌の河童はなにかの拍子にふとこの雄の河童を見ると｢大変です! 助けてください! あの河童はわたしを殺そうとするのです!｣と金切り声を出して叫びました｡もちろん大きい雄の河童はたちまち小さい河童をつかまえ､往来のまん中へねじ伏せました｡小さい河童は水掻きのある手に二三度空をつかんだなり､とうとう死んでしまいました｡けれどももうその時には雌の河童はにやにやしながら､大きい河童の頸っ玉へしっかりしがみついてしまっていたのです｡
 僕の知っていた雄の河童はだれも皆言い合わせたように雌の河童に追いかけられました｡もちろん妻子を持っているﾊﾞｯｸﾞでもやはり追いかけられたのです｡のみならず二三度はつかまったのです｡ただﾏｯｸﾞという哲学者だけは(これはあのﾄｯｸという詩人の隣にいる河童です｡)一度もつかまったことはありません｡これは一つにはﾏｯｸﾞぐらい､醜い河童も少ないためでしょう｡しかしまた一つにはﾏｯｸﾞだけはあまり往来へ顔を出さずに家にばかりいるためです｡僕はこのﾏｯｸﾞの家へも時々話しに出かけました｡ﾏｯｸﾞはいつも薄暗い部屋に七色の色硝子のﾗﾝﾀｱﾝをともし､脚の高い机に向かいながら､厚い本ばかり読んでいるのです｡僕はある時こういうﾏｯｸﾞと河童の恋愛を論じ合いました｡
｢なぜ政府は雌の河童が雄の河童を追いかけるのをもっと厳重に取り締まらないのです?｣
｢それは一つには官吏の中に雌の河童の少ないためですよ｡雌の河童は雄の河童よりもいっそう嫉妬心は強いものですからね､雌の河童の官吏さえ殖えれば､きっと今よりも雄の河童は追いかけられずに暮らせるでしょう｡しかしその効力もしれたものですね｡なぜと言ってごらんなさい｡官吏同志でも雌の河童は雄の河童を追いかけますからね｡｣
｢じゃあなたのように暮らしているのは一番幸福なわけですね｡｣
 するとﾏｯｸﾞは椅子を離れ､僕の両手を握ったまま､ため息といっしょにこう言いました｡
｢あなたは我々河童ではありませんから､おわかりにならないのももっともです｡しかしわたしもどうかすると､あの恐ろしい雌の河童に追いかけられたい気も起こるのですよ｡｣
七
 僕はまた詩人のﾄｯｸとたびたび音楽会へも出かけました｡が､いまだに忘れられないのは三度目に聴きにいった音楽会のことです｡もっとも会場の容子などはあまり日本と変わっていません｡やはりだんだんせり上がった席に雌雄の河童が三四百匹､いずれもﾌﾟﾛｸﾞﾗﾑを手にしながら､一心に耳を澄ませているのです｡僕はこの三度目の音楽会の時にはﾄｯｸやﾄｯｸの雌の河童のほかにも哲学者のﾏｯｸﾞといっしょになり､一番前の席にすわっていました｡するとｾﾛの独奏が終わった後､妙に目の細い河童が一匹､無造作に譜本を抱えたまま､壇の上へ上がってきました｡この河童はﾌﾟﾛｸﾞﾗﾑの教えるとおり､名高いｸﾗﾊﾞｯｸという作曲家です｡ﾌﾟﾛｸﾞﾗﾑの教えるとおり､――いや､ﾌﾟﾛｸﾞﾗﾑを見るまでもありません｡ｸﾗﾊﾞｯｸはﾄｯｸが属している超人倶楽部の会員ですから､僕もまた顔だけは知っているのです｡
｢Lied――Craback｣(この国のﾌﾟﾛｸﾞﾗﾑもたいていは独逸語を並べていました｡)
 ｸﾗﾊﾞｯｸは盛んな拍手のうちにちょっと我々へ一礼した後､静かにﾋﾟｱﾉの前へ歩み寄りました｡それからやはり無造作に自作のﾘｲﾄﾞを弾きはじめました｡ｸﾗﾊﾞｯｸはﾄｯｸの言葉によれば､この国の生んだ音楽家中､前後に比類のない天才だそうです｡僕はｸﾗﾊﾞｯｸの音楽はもちろん､そのまた余技の抒情詩にも興味を持っていましたから､大きい弓なりのﾋﾟｱﾉの音に熱心に耳を傾けていました｡ﾄｯｸやﾏｯｸﾞも恍惚としていたことはあるいは僕よりもまさっていたでしょう｡が､あの美しい(少なくとも河童たちの話によれば)雌の河童だけはしっかりﾌﾟﾛｸﾞﾗﾑを握ったなり､時々さもいらだたしそうに長い舌をべろべろ出していました｡これはﾏｯｸﾞの話によれば､なんでもかれこれ十年前にｸﾗﾊﾞｯｸをつかまえそこなったものですから､いまだにこの音楽家を目の敵にしているのだとかいうことです｡
 ｸﾗﾊﾞｯｸは全身に情熱をこめ､戦うようにﾋﾟｱﾉを弾きつづけました｡すると突然会場の中に神鳴りのように響き渡ったのは｢演奏禁止｣という声です｡僕はこの声にびっくりし､思わず後ろをふり返りました｡声の主は紛れもない､一番後ろの席にいる身の丈抜群の巡査です､巡査は僕がふり向いた時､悠然と腰をおろしたまま､もう一度前よりもおお声に｢演奏禁止｣と怒鳴りました｡それから､――
 それから先は大混乱です｡｢警官横暴!｣｢ｸﾗﾊﾞｯｸ､弾け! 弾け!｣｢莫迦!｣｢畜生!｣｢ひっこめ!｣｢負けるな!｣――こういう声のわき上がった中に椅子は倒れる､ﾌﾟﾛｸﾞﾗﾑは飛ぶ､おまけにだれが投げるのか､ｻｲﾀﾞｱの空罎や石ころやかじりかけの胡瓜さえ降ってくるのです｡僕は呆っ気にとられましたから､ﾄｯｸにその理由を尋ねようとしました｡が､ﾄｯｸも興奮したとみえ､椅子の上に突っ立ちながら､｢ｸﾗﾊﾞｯｸ､弾け! 弾け!｣とわめきつづけています｡のみならずﾄｯｸの雌の河童もいつの間に敵意を忘れたのか､｢警官横暴｣と叫んでいることは少しもﾄｯｸに変わりません｡僕はやむを得ずﾏｯｸﾞに向かい､｢どうしたのです?｣と尋ねてみました｡
｢これですか? これはこの国ではよくあることですよ｡元来画だの文芸だのは……｣
 ﾏｯｸﾞは何か飛んでくるたびにちょっと頸を縮めながら､相変わらず静かに説明しました｡
｢元来画だの文芸だのはだれの目にも何を表わしているかはとにかくちゃんとわかるはずですから､この国では決して発売禁止や展覧禁止は行なわれません｡その代わりにあるのが演奏禁止です｡なにしろ音楽というものだけはどんなに風俗を壊乱する曲でも､耳のない河童にはわかりませんからね｡｣
｢しかしあの巡査は耳があるのですか?｣
｢さあ､それは疑問ですね｡たぶん今の旋律を聞いているうちに細君といっしょに寝ている時の心臓の鼓動でも思い出したのでしょう｡｣
 こういう間にも大騒ぎはいよいよ盛んになるばかりです｡ｸﾗﾊﾞｯｸはﾋﾟｱﾉに向かったまま､傲然と我々をふり返っていました｡が､いくら傲然としていても､いろいろのものの飛んでくるのはよけないわけにゆきません｡従ってつまり二三秒置きにせっかくの態度も変わったわけです｡しかしとにかくだいたいとしては大音楽家の威厳を保ちながら､細い目をすさまじくかがやかせていました｡僕は――僕ももちろん危険を避けるためにﾄｯｸを小楯にとっていたものです｡が､やはり好奇心に駆られ､熱心にﾏｯｸﾞと話しつづけました｡
｢そんな検閲は乱暴じゃありませんか?｣
｢なに､どの国の検閲よりもかえって進歩しているくらいですよ｡たとえばをごらんなさい｡現につい一月ばかり前にも､……｣
 ちょうどこう言いかけたとたんです｡ﾏｯｸﾞはあいにく脳天に空罎が落ちたものですから､quack(これはただ間投詞です)と一声叫んだぎり､とうとう気を失ってしまいました｡
八
 僕は硝子会社の社長のｹﾞｴﾙに不思議にも好意を持っていました｡ｹﾞｴﾙは資本家中の資本家です｡おそらくはこの国の河童の中でも､ｹﾞｴﾙほど大きい腹をした河童は一匹もいなかったのに違いありません｡しかし茘枝に似た細君や胡瓜に似た子どもを左右にしながら､安楽椅子にすわっているところはほとんど幸福そのものです｡僕は時々裁判官のﾍﾟｯﾌﾟや医者のﾁｬｯｸにつれられてｹﾞｴﾙ家の晩餐へ出かけました｡またｹﾞｴﾙの紹介状を持ってｹﾞｴﾙやｹﾞｴﾙの友人たちが多少の関係を持っているいろいろの工場も見て歩きました｡そのいろいろの工場の中でもことに僕におもしろかったのは書籍製造会社の工場です｡僕は年の若い河童の技師とこの工場の中へはいり､水力電気を動力にした､大きい機械をながめた時､今さらのように河童の国の機械工業の進歩に驚嘆しました｡なんでもそこでは一年間に七百万部の本を製造するそうです｡が､僕を驚かしたのは本の部数ではありません｡それだけの本を製造するのに少しも手数のかからないことです｡なにしろこの国では本を造るのにただ機械の漏斗形の口へ紙とｲﾝｸと灰色をした粉末とを入れるだけなのですから｡それらの原料は機械の中へはいると､ほとんど五分とたたないうちに菊版､四六版､菊半裁版などの無数の本になって出てくるのです｡僕は瀑のように流れ落ちるいろいろの本をながめながら､反り身になった河童の技師にその灰色の粉末はなんと言うものかと尋ねてみました｡すると技師は黒光りに光った機械の前にたたずんだまま､つまらなそうにこう返事をしました｡
｢これですか? これは驢馬の脳髄ですよ｡ええ､一度乾燥させてから､ざっと粉末にしただけのものです｡時価は一噸二三銭ですがね｡｣
 もちろんこういう工業上の奇蹟は書籍製造会社にばかり起こっているわけではありません｡絵画製造会社にも､音楽製造会社にも､同じように起こっているのです｡実際またｹﾞｴﾙの話によれば､この国では平均一か月に七八百種の機械が新案され､なんでもずんずん人手を待たずに大量生産が行なわれるそうです｡従ってまた職工の解雇されるのも四五万匹を下らないそうです｡そのくせまだこの国では毎朝新聞を読んでいても､一度も罷業という字に出会いません｡僕はこれを妙に思いましたから､ある時またﾍﾟｯﾌﾟやﾁｬｯｸとｹﾞｴﾙ家の晩餐に招かれた機会にこのことをなぜかと尋ねてみました｡
｢それはみんな食ってしまうのですよ｡｣
 食後の葉巻をくわえたｹﾞｴﾙはいかにも無造作にこう言いました｡しかし｢食ってしまう｣というのはなんのことだかわかりません｡すると鼻目金をかけたﾁｬｯｸは僕の不審を察したとみえ､横あいから説明を加えてくれました｡
｢その職工をみんな殺してしまって､肉を食料に使うのです｡ここにある新聞をごらんなさい｡今月はちょうど六万四千七百六十九匹の職工が解雇されましたから､それだけ肉の値段も下がったわけですよ｡｣
｢職工は黙って殺されるのですか?｣
｢それは騒いでもしかたはありません｡職工屠殺法があるのですから｡｣
 これは山桃の鉢植えを後ろに苦い顔をしていたﾍﾟｯﾌﾟの言葉です｡僕はもちろん不快を感じました｡しかし主人公のｹﾞｴﾙはもちろん､ﾍﾟｯﾌﾟやﾁｬｯｸもそんなことは当然と思っているらしいのです｡現にﾁｬｯｸは笑いながら､あざけるように僕に話しかけました｡
｢つまり餓死したり自殺したりする手数を国家的に省略してやるのですね｡ちょっと有毒瓦斯をかがせるだけですから､たいした苦痛はありませんよ｡｣
｢けれどもその肉を食うというのは､……｣
｢常談を言ってはいけません｡あのﾏｯｸﾞに聞かせたら､さぞ大笑いに笑うでしょう｡あなたの国でも第四階級の娘たちは売笑婦になっているではありませんか? 職工の肉を食うことなどに憤慨したりするのは感傷主義ですよ｡｣
 こういう問答を聞いていたｹﾞｴﾙは手近いﾃｴﾌﾞﾙの上にあったｻﾝﾄﾞｳｨｯﾁの皿を勧めながら､恬然と僕にこう言いました｡
｢どうです? 一つとりませんか? これも職工の肉ですがね｡｣
 僕はもちろん辟易しました｡いや､そればかりではありません｡ﾍﾟｯﾌﾟやﾁｬｯｸの笑い声を後ろにｹﾞｴﾙ家の客間を飛び出しました｡それはちょうど家々の空に星明かりも見えない荒れ模様の夜です｡僕はその闇の中を僕の住居へ帰りながら､のべつ幕なしに嘔吐を吐きました｡夜目にも白じらと流れる嘔吐を｡
九
 しかし硝子会社の社長のｹﾞｴﾙは人なつこい河童だったのに違いません｡僕はたびたびｹﾞｴﾙといっしょにｹﾞｴﾙの属している倶楽部へ行き､愉快に一晩を暮らしました｡これは一つにはその倶楽部はﾄｯｸの属している超人倶楽部よりもはるかに居心のよかったためです｡のみならずまたｹﾞｴﾙの話は哲学者のﾏｯｸﾞの話のように深みを持っていなかったにせよ､僕には全然新しい世界を､――広い世界をのぞかせました｡ｹﾞｴﾙは､いつも純金の匙に珈琲の茶碗をかきまわしながら､快活にいろいろの話をしたものです｡
 なんでもある霧の深い晩､僕は冬薔薇を盛った花瓶を中にｹﾞｴﾙの話を聞いていました｡それはたしか部屋全体はもちろん､椅子やﾃｴﾌﾞﾙも白い上に細い金の縁をとったｾｾｯｼｮﾝ風の部屋だったように覚えています｡ｹﾞｴﾙはふだんよりも得意そうに顔中に微笑をみなぎらせたまま､ちょうどそのころ天下を取っていた Quorax 党内閣のことなどを話しました｡ｸｵﾗｯｸｽという言葉はただ意味のない間投詞ですから､｢おや｣とでも訳すほかはありません｡が､とにかく何よりも先に｢河童全体の利益｣ということを標榜していた政党だったのです｡
｢ｸｵﾗｯｸｽ党を支配しているものは名高い政治家のﾛｯﾍﾟです｡『正直は最良の外交である』とはﾋﾞｽﾏﾙｸの言った言葉でしょう｡しかしﾛｯﾍﾟは正直を内治の上にも及ぼしているのです｡……｣
｢けれどもﾛｯﾍﾟの演説は……｣
｢まあ､わたしの言うことをお聞きなさい｡あの演説はもちろんことごとくｳｿです｡が､ｳｿということはだれでも知っていますから､畢竟正直と変わらないでしょう､それを一概にｳｿと言うのはあなたがただけの偏見ですよ｡我々河童はあなたがたのように､……しかしそれはどうでもよろしい｡わたしの話したいのはﾛｯﾍﾟのことです｡ﾛｯﾍﾟはｸｵﾗｯｸｽ党を支配している､そのまたﾛｯﾍﾟを支配しているものは Pou-Fou 新聞の(この『ﾌﾟｳ･ﾌｳ』という言葉もやはり意味のない間投詞です｡もし強いて訳すれば､『ああ』とでも言うほかはありません｡)社長のｸｲｸｲです｡が､ｸｲｸｲも彼自身の主人というわけにはゆきません｡ｸｲｸｲを支配しているものはあなたの前にいるｹﾞｴﾙです｡｣
｢けれども――これは失礼かもしれませんけれども､ﾌﾟｳ･ﾌｳ新聞は労働者の味かたをする新聞でしょう｡その社長のｸｲｸｲもあなたの支配を受けているというのは､……｣
｢ﾌﾟｳ･ﾌｳ新聞の記者たちはもちろん労働者の味かたです｡しかし記者たちを支配するものはｸｲｸｲのほかはありますまい｡しかもｸｲｸｲはこのｹﾞｴﾙの後援を受けずにはいられないのです｡｣
 ｹﾞｴﾙは相変わらず微笑しながら､純金の匙をおもちゃにしています｡僕はこういうｹﾞｴﾙを見ると､ｹﾞｴﾙ自身を憎むよりも､ﾌﾟｳ･ﾌｳ新聞の記者たちに同情の起こるのを感じました｡するとｹﾞｴﾙは僕の無言にたちまちこの同情を感じたとみえ､大きい腹をふくらませてこう言うのです｡
｢なに､ﾌﾟｳ･ﾌｳ新聞の記者たちも全部労働者の味かたではありませんよ｡少なくとも我々河童というものはだれの味かたをするよりも先に我々自身の味かたをしますからね｡……しかしさらに厄介なことにはこのｹﾞｴﾙ自身さえやはり他人の支配を受けているのです｡あなたはそれをだれだと思いますか? それはわたしの妻ですよ｡美しいｹﾞｴﾙ夫人ですよ｡｣
 ｹﾞｴﾙはおお声に笑いました｡
｢それはむしろしあわせでしょう｡｣
｢とにかくわたしは満足しています｡しかしこれもあなたの前だけに､――河童でないあなたの前だけに手放しで吹聴できるのです｡｣
｢するとつまりｸｵﾗｯｸｽ内閣はｹﾞｴﾙ夫人が支配しているのですね｡｣
｢さあそうも言われますかね｡……しかし七年前の戦争などはたしかにある雌の河童のために始まったものに違いありません｡｣
｢戦争? この国にも戦争はあったのですか?｣
｢ありましたとも｡将来もいつあるかわかりません｡なにしろ隣国のある限りは､……｣
 僕は実際この時はじめて河童の国も国家的に孤立していないことを知りました｡ｹﾞｴﾙの説明するところによれば､河童はいつも獺を仮設敵にしているということです｡しかも獺は河童に負けない軍備を具えているということです｡僕はこの獺を相手に河童の戦争した話に少なからず興味を感じました｡(なにしろ河童の強敵に獺のいるなどということは｢水虎考略｣の著者はもちろん､｢山島民譚集｣の著者柳田国男さんさえ知らずにいたらしい新事実ですから｡)
｢あの戦争の起こる前にはもちろん両国とも油断せずにじっと相手をうかがっていました｡というのはどちらも同じように相手を恐怖していたからです｡そこへこの国にいた獺が一匹､ある河童の夫婦を訪問しました｡そのまた雌の河童というのは亭主を殺すつもりでいたのです｡なにしろ亭主は道楽者でしたからね｡おまけに生命保険のついていたことも多少の誘惑になったかもしれません｡｣
｢あなたはその夫婦を御存じですか?｣
｢ええ､――いや､雄の河童だけは知っています｡わたしの妻などはこの河童を悪人のように言っていますがね｡しかしわたしに言わせれば､悪人よりもむしろ雌の河童につかまることを恐れている被害妄想の多い狂人です｡……そこでこの雌の河童は亭主のｺｺｱの茶碗の中へ青化加里を入れておいたのです｡それをまたどう間違えたか､客の獺に飲ませてしまったのです｡獺はもちろん死んでしまいました｡それから……｣
｢それから戦争になったのですか?｣
｢ええ､あいにくその獺は勲章を持っていたものですからね｡｣
｢戦争はどちらの勝ちになったのですか?｣
｢もちろんこの国の勝ちになったのです｡三十六万九千五百匹の河童たちはそのために健気にも戦死しました｡しかし敵国に比べれば､そのくらいの損害はなんともありません｡この国にある毛皮という毛皮はたいてい獺の毛皮です｡わたしもあの戦争の時には硝子を製造するほかにも石炭殻を戦地へ送りました｡｣
｢石炭殻を何にするのですか?｣
｢もちろん食糧にするのです｡我々は､河童は腹さえ減れば､なんでも食うのにきまっていますからね｡｣
｢それは――どうか怒らずにください｡それは戦地にいる河童たちには……我々の国では醜聞ですがね｡｣
｢この国でも醜聞には違いありません｡しかしわたし自身こう言っていれば､だれも醜聞にはしないものです｡哲学者のﾏｯｸﾞも言っているでしょう｡『汝の悪は汝自ら言え｡悪はおのずから消滅すべし｡』……しかもわたしは利益のほかにも愛国心に燃え立っていたのですからね｡｣
 ちょうどそこへはいってきたのはこの倶楽部の給仕です｡給仕はｹﾞｴﾙにお時宜をした後､朗読でもするようにこう言いました｡
｢お宅のお隣に火事がございます｡｣
｢火――火事!｣
 ｹﾞｴﾙは驚いて立ち上がりました｡僕も立ち上がったのはもちろんです｡が､給仕は落ち着き払って次の言葉をつけ加えました｡
｢しかしもう消し止めました｡｣
 ｹﾞｴﾙは給仕を見送りながら､泣き笑いに近い表情をしました｡僕はこういう顔を見ると､いつかこの硝子会社の社長を憎んでいたことに気づきました｡が､ｹﾞｴﾙはもう今では大資本家でもなんでもないただの河童になって立っているのです｡僕は花瓶の中の冬薔薇の花を抜き､ｹﾞｴﾙの手へ渡しました｡
｢しかし火事は消えたといっても､奥さんはさぞお驚きでしょう｡さあ､これを持ってお帰りなさい｡｣
｢ありがとう｡｣
 ｹﾞｴﾙは僕の手を握りました｡それから急ににやりと笑い､小声にこう僕に話しかけました｡
｢隣はわたしの家作ですからね｡火災保険の金だけはとれるのですよ｡｣
 僕はこの時のｹﾞｴﾙの微笑を――軽蔑することもできなければ､憎悪することもできないｹﾞｴﾙの微笑をいまだにありありと覚えています｡
十
｢どうしたね? きょうはまた妙にふさいでいるじゃないか?｣
 その火事のあった翌日です｡僕は巻煙草をくわえながら､僕の客間の椅子に腰をおろした学生のﾗｯﾌﾟにこう言いました｡実際またﾗｯﾌﾟは右の脚の上へ左の脚をのせたまま､腐った嘴も見えないほど､ぼんやり床の上ばかり見ていたのです｡
｢ﾗｯﾌﾟ君､どうしたね｡｣と言えば､
｢いや､なに､つまらないことなのですよ｡――｣
 ﾗｯﾌﾟはやっと頭をあげ､悲しい鼻声を出しました｡
｢僕はきょう窓の外を見ながら､『おや虫取り菫が咲いた』と何気なしにつぶやいたのです｡すると僕の妹は急に顔色を変えたと思うと､『どうせわたしは虫取り菫よ』と当たり散らすじゃありませんか? おまけにまた僕のおふくろも大の妹贔屓ですから､やはり僕に食ってかかるのです｡｣
｢虫取り菫が咲いたということはどうして妹さんには不快なのだね?｣
｢さあ､たぶん雄の河童をつかまえるという意味にでもとったのでしょう｡そこへおふくろと仲悪い叔母も喧嘩の仲間入りをしたのですから､いよいよ大騒動になってしまいました｡しかも年中酔っ払っているおやじはこの喧嘩を聞きつけると､たれかれの差別なしに殴り出したのです｡それだけでも始末のつかないところへ僕の弟はその間におふくろの財布を盗むが早いか､ｷﾈﾏか何かを見にいってしまいました｡僕は……ほんとうに僕はもう､……｣
 ﾗｯﾌﾟは両手に顔を埋め､何も言わずに泣いてしまいました｡僕の同情したのはもちろんです｡同時にまた家族制度に対する詩人のﾄｯｸの軽蔑を思い出したのももちろんです｡僕はﾗｯﾌﾟの肩をたたき､一生懸命に慰めました｡
｢そんなことはどこでもありがちだよ｡まあ勇気を出したまえ｡｣
｢しかし……しかし嘴でも腐っていなければ､……｣
｢それはあきらめるほかはないさ｡さあ､ﾄｯｸ君の家へでも行こう｡｣
｢ﾄｯｸさんは僕を軽蔑しています｡僕はﾄｯｸさんのように大胆に家族を捨てることができませんから｡｣
｢じゃｸﾗﾊﾞｯｸ君の家へ行こう｡｣
 僕はあの音楽会以来､ｸﾗﾊﾞｯｸにも友だちになっていましたから､とにかくこの大音楽家の家へﾗｯﾌﾟをつれ出すことにしました｡ｸﾗﾊﾞｯｸはﾄｯｸに比べれば､はるかに贅沢に暮らしています｡というのは資本家のｹﾞｴﾙのように暮らしているという意味ではありません｡ただいろいろの骨董を､――ﾀﾅｸﾞﾗの人形やﾍﾟﾙｼｱの陶器を部屋いっぱいに並べた中にﾄﾙｺ風の長椅子を据え､ｸﾗﾊﾞｯｸ自身の肖像画の下にいつも子どもたちと遊んでいるのです｡が､きょうはどうしたのか両腕を胸へ組んだまま､苦い顔をしてすわっていました｡のみならずそのまた足もとには紙屑が一面に散らばっていました｡ﾗｯﾌﾟも詩人ﾄｯｸといっしょにたびたびｸﾗﾊﾞｯｸには会っているはずです｡しかしこの容子に恐れたとみえ､きょうは丁寧にお時宜をしたなり､黙って部屋の隅に腰をおろしました｡
｢どうしたね? ｸﾗﾊﾞｯｸ君｡｣
 僕はほとんど挨拶の代わりにこう大音楽家へ問いかけました｡
｢どうするものか? 批評家の阿呆め! 僕の抒情詩はﾄｯｸの抒情詩と比べものにならないと言やがるんだ｡｣
｢しかし君は音楽家だし､……｣
｢それだけならば我慢もできる｡僕はﾛｯｸに比べれば､音楽家の名に価しないと言やがるじゃないか?｣
 ﾛｯｸというのはｸﾗﾊﾞｯｸとたびたび比べられる音楽家です｡が､あいにく超人倶楽部の会員になっていない関係上､僕は一度も話したことはありません｡もっとも嘴の反り上がった､一癖あるらしい顔だけはたびたび写真でも見かけていました｡
｢ﾛｯｸも天才には違いない｡しかしﾛｯｸの音楽は君の音楽にあふれている近代的情熱を持っていない｡｣
｢君はほんとうにそう思うか?｣
｢そう思うとも｡｣
 するとｸﾗﾊﾞｯｸは立ち上がるが早いか､ﾀﾅｸﾞﾗの人形をひっつかみ､いきなり床の上にたたきつけました｡ﾗｯﾌﾟはよほど驚いたとみえ､何か声をあげて逃げようとしました｡が､ｸﾗﾊﾞｯｸはﾗｯﾌﾟや僕にはちょっと｢驚くな｣という手真似をした上､今度は冷やかにこう言うのです｡
｢それは君もまた俗人のように耳を持っていないからだ｡僕はﾛｯｸを恐れている｡……｣
｢君が? 謙遜家を気どるのはやめたまえ｡｣
｢だれが謙遜家を気どるものか? 第一君たちに気どって見せるくらいならば､批評家たちの前に気どって見せている｡僕は――ｸﾗﾊﾞｯｸは天才だ｡その点ではﾛｯｸを恐れていない｡｣
｢では何を恐れているのだ?｣
｢何か正体の知れないものを､――言わばﾛｯｸを支配している星を｡｣
｢どうも僕には腑に落ちないがね｡｣
｢ではこう言えばわかるだろう｡ﾛｯｸは僕の影響を受けない｡が､僕はいつの間にかﾛｯｸの影響を受けてしまうのだ｡｣
｢それは君の感受性の……｡｣
｢まあ､聞きたまえ｡感受性などの問題ではない｡ﾛｯｸはいつも安んじてあいつだけにできる仕事をしている｡しかし僕はいらいらするのだ｡それはﾛｯｸの目から見れば､あるいは一歩の差かもしれない｡けれども僕には十哩も違うのだ｡｣
｢しかし先生の英雄曲は……｣
 ｸﾗﾊﾞｯｸは細い目をいっそう細め､いまいましそうにﾗｯﾌﾟをにらみつけました｡
｢黙りたまえ｡君などに何がわかる? 僕はﾛｯｸを知っているのだ｡ﾛｯｸに平身低頭する犬どもよりもﾛｯｸを知っているのだ｡｣
｢まあ少し静かにしたまえ｡｣
｢もし静かにしていられるならば､……僕はいつもこう思っている｡――僕らの知らない何ものかは僕を､――ｸﾗﾊﾞｯｸをあざけるためにﾛｯｸを僕の前に立たせたのだ｡哲学者のﾏｯｸﾞはこういうことをなにもかも承知している｡いつもあの色硝子のﾗﾝﾀｱﾝの下に古ぼけた本ばかり読んでいるくせに｡｣
｢どうして?｣
｢この近ごろﾏｯｸﾞの書いた『阿呆の言葉』という本を見たまえ｡――｣
 ｸﾗﾊﾞｯｸは僕に一冊の本を渡す――というよりも投げつけました｡それからまた腕を組んだまま､突けんどんにこう言い放ちました｡
｢じゃきょうは失敬しよう｡｣
 僕はしょげ返ったﾗｯﾌﾟといっしょにもう一度往来へ出ることにしました｡人通りの多い往来は相変わらず毛生欅の並み木のかげにいろいろの店を並べています｡僕らはなんということもなしに黙って歩いてゆきました｡するとそこへ通りかかったのは髪の長い詩人のﾄｯｸです｡ﾄｯｸは僕らの顔を見ると､腹の袋から手巾を出し､何度も額をぬぐいました｡
｢やあ､しばらく会わなかったね｡僕はきょうは久しぶりにｸﾗﾊﾞｯｸを尋ねようと思うのだが､……｣
 僕はこの芸術家たちを喧嘩させては悪いと思い､ｸﾗﾊﾞｯｸのいかにも不機嫌だったことを婉曲にﾄｯｸに話しました｡
｢そうか｡じゃやめにしよう｡なにしろｸﾗﾊﾞｯｸは神経衰弱だからね｡……僕もこの二三週間は眠られないのに弱っているのだ｡｣
｢どうだね､僕らといっしょに散歩をしては?｣
｢いや､きょうはやめにしよう｡おや!｣
 ﾄｯｸはこう叫ぶが早いか､しっかり僕の腕をつかみました｡しかもいつか体中に冷汗を流しているのです｡
｢どうしたのだ?｣
｢どうしたのです?｣
｢なにあの自動車の窓の中から緑いろの猿が一匹首を出したように見えたのだよ｡｣
 僕は多少心配になり､とにかくあの医者のﾁｬｯｸに診察してもらうように勧めました｡しかしﾄｯｸはなんと言っても､承知する気色さえ見せません｡のみならず何か疑わしそうに僕らの顔を見比べながら､こんなことさえ言い出すのです｡
｢僕は決して無政府主義者ではないよ｡それだけはきっと忘れずにいてくれたまえ｡――ではさようなら｡ﾁｬｯｸなどはまっぴらごめんだ｡｣
 僕らはぼんやりたたずんだまま､ﾄｯｸの後ろ姿を見送っていました｡僕らは――いや､｢僕ら｣ではありません｡学生のﾗｯﾌﾟはいつの間にか往来のまん中に脚をひろげ､しっきりない自動車や人通りを股目金にのぞいているのです｡僕はこの河童も発狂したかと思い､驚いてﾗｯﾌﾟを引き起こしました｡
｢常談じゃない｡何をしている?｣
 しかしﾗｯﾌﾟは目をこすりながら､意外にも落ち着いて返事をしました｡
｢いえ､あまり憂鬱ですから､さかさまに世の中をながめて見たのです｡けれどもやはり同じことですね｡｣
十一
 これは哲学者のﾏｯｸﾞの書いた｢阿呆の言葉｣の中の何章かです｡――
        
 阿呆はいつも彼以外のものを阿呆であると信じている｡
        
 我々の自然を愛するのは自然は我々を憎んだり嫉妬したりしないためもないことはない｡
        
 もっとも賢い生活は一時代の習慣を軽蔑しながら､しかもそのまた習慣を少しも破らないように暮らすことである｡
        
 我々のもっとも誇りたいものは我々の持っていないものだけである｡
        
 何びとも偶像を破壊することに異存を持っているものはない｡同時にまた何びとも偶像になることに異存を持っているものはない｡しかし偶像の台座の上に安んじてすわっていられるものはもっとも神々に恵まれたもの､――阿呆か､悪人か､英雄かである｡(ｸﾗﾊﾞｯｸはこの章の上へ爪の痕をつけていました｡)
        
 我々の生活に必要な思想は三千年前に尽きたかもしれない｡我々はただ古い薪に新しい炎を加えるだけであろう｡
        
 我々の特色は我々自身の意識を超越するのを常としている｡
        
 幸福は苦痛を伴い､平和は倦怠を伴うとすれば､――?
        
 自己を弁護することは他人を弁護することよりも困難である｡疑うものは弁護士を見よ｡
        
 矜誇[#ﾙﾋﾞの｢きょうか｣はﾏﾏ]､愛欲､疑惑――あらゆる罪は三千年来､この三者から発している｡同時にまたおそらくはあらゆる徳も｡
        
 物質的欲望を減ずることは必ずしも平和をもたらさない｡我々は平和を得るためには精神的欲望も減じなければならぬ｡(ｸﾗﾊﾞｯｸはこの章の上にも爪の痕を残していました｡)
        
 我々は人間よりも不幸である｡人間は河童ほど進化していない｡(僕はこの章を読んだ時思わず笑ってしまいました｡)
        
 成すことは成し得ることであり､成し得ることは成すことである｡畢竟我々の生活はこういう循環論法を脱することはできない｡――すなわち不合理に終始している｡
        
 ﾎﾞｵﾄﾞﾚｴﾙは白痴になった後､彼の人生観をたった一語に､――女陰の一語に表白した｡しかし彼自身を語るものは必ずしもこう言ったことではない｡むしろ彼の天才に､――彼の生活を維持するに足る詩的天才に信頼したために胃袋の一語を忘れたことである｡(この章にもやはりｸﾗﾊﾞｯｸの爪の痕は残っていました｡)
        
 もし理性に終始するとすれば､我々は当然我々自身の存在を否定しなければならぬ｡理性を神にしたｳﾞｫﾙﾃｴﾙの幸福に一生をおわったのはすなわち人間の河童よりも進化していないことを示すものである｡
十二
 ある割合に寒い午後です｡僕は｢阿呆の言葉｣を読み飽きましたから､哲学者のﾏｯｸﾞを尋ねに出かけました｡するとある寂しい町の角に蚊のようにやせた河童が一匹､ぼんやり壁によりかかっていました｡しかもそれは紛れもない､いつか僕の万年筆を盗んでいった河童なのです｡僕はしめたと思いましたから､ちょうどそこへ通りかかった､たくましい巡査を呼びとめました｡
｢ちょっとあの河童を取り調べてください｡あの河童はちょうど一月ばかり前にわたしの万年筆を盗んだのですから｡｣
 巡査は右手の棒をあげ､(この国の巡査は剣の代わりに水松の棒を持っているのです｡)｢おい､君｣とその河童へ声をかけました｡僕はあるいはその河童は逃げ出しはしないかと思っていました｡が､存外落ち着き払って巡査の前へ歩み寄りました｡のみならず腕を組んだまま､いかにも傲然と僕の顔や巡査の顔をじろじろ見ているのです｡しかし巡査は怒りもせず､腹の袋から手帳を出してさっそく尋問にとりかかりました｡
｢お前の名は?｣
｢ｸﾞﾙｯｸ｡｣
｢職業は?｣
｢つい二三日前までは郵便配達夫をしていました｡｣
｢よろしい｡そこでこの人の申し立てによれば､君はこの人の万年筆を盗んでいったということだがね｡｣
｢ええ､一月ばかり前に盗みました｡｣
｢なんのために?｣
｢子どもの玩具にしようと思ったのです｡｣
｢その子どもは?｣
 巡査ははじめて相手の河童へ鋭い目を注ぎました｡
｢一週間前に死んでしまいました｡｣
｢死亡証明書を持っているかね?｣
 やせた河童は腹の袋から一枚の紙をとり出しました｡巡査はその紙へ目を通すと､急ににやにや笑いながら､相手の肩をたたきました｡
｢よろしい｡どうも御苦労だったね｡｣
 僕は呆気にとられたまま､巡査の顔をながめていました｡しかもそのうちにやせた河童は何かぶつぶつつぶやきながら､僕らを後ろにして行ってしまうのです｡僕はやっと気をとり直し､こう巡査に尋ねてみました｡
｢どうしてあの河童をつかまえないのです?｣
｢あの河童は無罪ですよ｡｣
｢しかし僕の万年筆を盗んだのは……｣
｢子どもの玩具にするためだったのでしょう｡けれどもその子どもは死んでいるのです｡もし何か御不審だったら､刑法千二百八十五条をお調べなさい｡｣
 巡査はこう言いすてたなり､さっさとどこかへ行ってしまいました｡僕はしかたがありませんから､｢刑法千二百八十五条｣を口の中に繰り返し､ﾏｯｸﾞの家へ急いでゆきました｡哲学者のﾏｯｸﾞは客好きです｡現にきょうも薄暗い部屋には裁判官のﾍﾟｯﾌﾟや医者のﾁｬｯｸや硝子会社の社長のｹﾞｴﾙなどが集まり､七色の色硝子のﾗﾝﾀｱﾝの下に煙草の煙を立ち昇らせていました｡そこに裁判官のﾍﾟｯﾌﾟが来ていたのは何よりも僕には好つごうです｡僕は椅子にかけるが早いか､刑法第千二百八十五条を検べる代わりにさっそくﾍﾟｯﾌﾟへ問いかけました｡
｢ﾍﾟｯﾌﾟ君､はなはだ失礼ですが､この国では罪人を罰しないのですか?｣
 ﾍﾟｯﾌﾟは金口の煙草の煙をまず悠々と吹き上げてから､いかにもつまらなそうに返事をしました｡
｢罰しますとも｡死刑さえ行なわれるくらいですからね｡｣
｢しかし僕は一月ばかり前に､……｣
 僕は委細を話した後､例の刑法千二百八十五条のことを尋ねてみました｡
｢ふむ､それはこういうのです｡――『いかなる犯罪を行ないたりといえども､該犯罪を行なわしめたる事情の消失したる後は該犯罪者を処罰することを得ず』つまりあなたの場合で言えば､その河童はかつては親だったのですが､今はもう親ではありませんから､犯罪も自然と消滅するのです｡｣
｢それはどうも不合理ですね｡｣
｢常談を言ってはいけません｡親だった河童も親である河童も同一に見るのこそ不合理です｡そうそう､日本の法律では同一に見ることになっているのですね｡それはどうも我々には滑稽です｡ふふふふふふふふふふ｡｣
 ﾍﾟｯﾌﾟは巻煙草をほうり出しながら､気のない薄笑いをもらしていました｡そこへ口を出したのは法律には縁の遠いﾁｬｯｸです｡ﾁｬｯｸはちょっと鼻目金を直し､こう僕に質問しました｡
｢日本にも死刑はありますか?｣
｢ありますとも｡日本では絞罪です｡｣
 僕は冷然と構えこんだﾍﾟｯﾌﾟに多少反感を感じていましたから､この機会に皮肉を浴びせてやりました｡
｢この国の死刑は日本よりも文明的にできているでしょうね?｣
｢それはもちろん文明的です｡｣
 ﾍﾟｯﾌﾟはやはり落ち着いていました｡
｢この国では絞罪などは用いません｡まれには電気を用いることもあります｡しかしたいていは電気も用いません｡ただその犯罪の名を言って聞かせるだけです｡｣
｢それだけで河童は死ぬのですか?｣
｢死にますとも｡我々河童の神経作用はあなたがたのよりも微妙ですからね｡｣
｢それは死刑ばかりではありません｡殺人にもその手を使うのがあります――｣
 社長のｹﾞｴﾙは色硝子の光に顔中紫に染まりながら､人なつこい笑顔をして見せました｡
｢わたしはこの間もある社会主義者に『貴様は盗人だ』と言われたために心臓痲痺を起こしかかったものです｡｣
｢それは案外多いようですね｡わたしの知っていたある弁護士などはやはりそのために死んでしまったのですからね｡｣
 僕はこう口を入れた河童､――哲学者のﾏｯｸﾞをふりかえりました｡ﾏｯｸﾞはやはりいつものように皮肉な微笑を浮かべたまま､だれの顔も見ずにしゃべっているのです｡
｢その河童はだれかに蛙だと言われ､――もちろんあなたも御承知でしょう､この国で蛙だと言われるのは人非人という意味になることぐらいは｡――己は蛙かな? 蛙ではないかな? と毎日考えているうちにとうとう死んでしまったものです｡｣
｢それはつまり自殺ですね｡｣
｢もっともその河童を蛙だと言ったやつは殺すつもりで言ったのですがね｡あなたがたの目から見れば､やはりそれも自殺という……｣
 ちょうどﾏｯｸﾞがこう言った時です｡突然その部屋の壁の向こうに､――たしかに詩人のﾄｯｸの家に鋭いﾋﾟｽﾄﾙの音が一発､空気をはね返すように響き渡りました｡
十三
 僕らはﾄｯｸの家へ駆けつけました｡ﾄｯｸは右の手にﾋﾟｽﾄﾙを握り､頭の皿から血を出したまま､高山植物の鉢植えの中に仰向けになって倒れていました｡そのまたそばには雌の河童が一匹､ﾄｯｸの胸に顔を埋め､大声をあげて泣いていました｡僕は雌の河童を抱き起こしながら､(いったい僕はぬらぬらする河童の皮膚に手を触れることをあまり好んではいないのですが｡)｢どうしたのです?｣と尋ねました｡
｢どうしたのだか､わかりません｡ただ何か書いていたと思うと､いきなりﾋﾟｽﾄﾙで頭を打ったのです｡ああ､わたしはどうしましょう? qur-r-r-r-r, qur-r-r-r-r｣(これは河童の泣き声です｡)
｢なにしろﾄｯｸ君はわがままだったからね｡｣
 硝子会社の社長のｹﾞｴﾙは悲しそうに頭を振りながら､裁判官のﾍﾟｯﾌﾟにこう言いました｡しかしﾍﾟｯﾌﾟは何も言わずに金口の巻煙草に火をつけていました｡すると今までひざまずいて､ﾄｯｸの創口などを調べていたﾁｬｯｸはいかにも医者らしい態度をしたまま､僕ら五人に宣言しました｡(実はひとりと四匹とです｡)
｢もう駄目です｡ﾄｯｸ君は元来胃病でしたから､それだけでも憂鬱になりやすかったのです｡｣
｢何か書いていたということですが｡｣
 哲学者のﾏｯｸﾞは弁解するようにこう独り語をもらしながら､机の上の紙をとり上げました｡僕らは皆頸をのばし､(もっとも僕だけは例外です｡)幅の広いﾏｯｸﾞの肩越しに一枚の紙をのぞきこみました｡
｢いざ､立ちてゆかん｡娑婆界を隔つる谷へ｡
 岩むらはこごしく､やま水は清く､
 薬草の花はにおえる谷へ｡｣
 ﾏｯｸﾞは僕らをふり返りながら､微苦笑といっしょにこう言いました｡
｢これはｹﾞｴﾃの『ﾐﾆﾖﾝの歌』の剽窃ですよ｡するとﾄｯｸ君の自殺したのは詩人としても疲れていたのですね｡｣
 そこへ偶然自動車を乗りつけたのはあの音楽家のｸﾗﾊﾞｯｸです｡ｸﾗﾊﾞｯｸはこういう光景を見ると､しばらく戸口にたたずんでいました｡が､僕らの前へ歩み寄ると､怒鳴りつけるようにﾏｯｸﾞに話しかけました｡
｢それはﾄｯｸの遺言状ですか?｣
｢いや､最後に書いていた詩です｡｣
｢詩?｣
 やはり少しも騒がないﾏｯｸﾞは髪を逆立てたｸﾗﾊﾞｯｸにﾄｯｸの詩稿を渡しました｡ｸﾗﾊﾞｯｸはあたりには目もやらずに熱心にその詩稿を読み出しました｡しかもﾏｯｸﾞの言葉にはほとんど返事さえしないのです｡
｢あなたはﾄｯｸ君の死をどう思いますか?｣
｢いざ､立ちて､……僕もまたいつ死ぬかわかりません｡……娑婆界を隔つる谷へ｡……｣
｢しかしあなたはﾄｯｸ君とはやはり親友のひとりだったのでしょう?｣
｢親友? ﾄｯｸはいつも孤独だったのです｡……娑婆界を隔つる谷へ､……ただﾄｯｸは不幸にも､……岩むらはこごしく……｣
｢不幸にも?｣
｢やま水は清く､……あなたがたは幸福です｡……岩むらはこごしく｡……｣
 僕はいまだに泣き声を絶たない雌の河童に同情しましたから､そっと肩を抱えるようにし､部屋の隅の長椅子へつれていきました｡そこには二歳か三歳かの河童が一匹､何も知らずに笑っているのです｡僕は雌の河童の代わりに子どもの河童をあやしてやりました｡するといつか僕の目にも涙のたまるのを感じました｡僕が河童の国に住んでいるうちに涙というものをこぼしたのは前にもあとにもこの時だけです｡
｢しかしこういうわがままの河童といっしょになった家族は気の毒ですね｡｣
｢なにしろあとのことも考えないのですから｡｣
 裁判官のﾍﾟｯﾌﾟは相変わらず､新しい巻煙草に火をつけながら､資本家のｹﾞｴﾙに返事をしていました｡すると僕らを驚かせたのは音楽家のｸﾗﾊﾞｯｸのおお声です｡ｸﾗﾊﾞｯｸは詩稿を握ったまま､だれにともなしに呼びかけました｡
｢しめた! すばらしい葬送曲ができるぞ｡｣
 ｸﾗﾊﾞｯｸは細い目をかがやかせたまま､ちょっとﾏｯｸﾞの手を握ると､いきなり戸口へ飛んでいきました｡もちろんもうこの時には隣近所の河童が大勢､ﾄｯｸの家の戸口に集まり､珍しそうに家の中をのぞいているのです｡しかしｸﾗﾊﾞｯｸはこの河童たちを遮二無二左右へ押しのけるが早いか､ひらりと自動車へ飛び乗りました｡同時にまた自動車は爆音を立ててたちまちどこかへ行ってしまいました｡
｢こら､こら､そうのぞいてはいかん｡｣
 裁判官のﾍﾟｯﾌﾟは巡査の代わりに大勢の河童を押し出した後､ﾄｯｸの家の戸をしめてしまいました｡部屋の中はそのせいか急にひっそりなったものです｡僕らはこういう静かさの中に――高山植物の花の香に交じったﾄｯｸの血の匂いの中に後始末のことなどを相談しました｡しかしあの哲学者のﾏｯｸﾞだけはﾄｯｸの死骸をながめたまま､ぼんやり何か考えています｡僕はﾏｯｸﾞの肩をたたき､｢何を考えているのです?｣と尋ねました｡
｢河童の生活というものをね｡｣
｢河童の生活がどうなるのです?｣
｢我々河童はなんと言っても､河童の生活をまっとうするためには､……｣
 ﾏｯｸﾞは多少はずかしそうにこう小声でつけ加えました｡
｢とにかく我々河童以外の何ものかの力を信ずることですね｡｣
一四
 僕に宗教というものを思い出させたのはこういうﾏｯｸﾞの言葉です｡僕はもちろん物質主義者ですから､真面目に宗教を考えたことは一度もなかったのに違いありません｡が､この時はﾄｯｸの死にある感動を受けていたためにいったい河童の宗教はなんであるかと考え出したのです｡僕はさっそく学生のﾗｯﾌﾟにこの問題を尋ねてみました｡
｢それは基督教､仏教､ﾓﾊﾒｯﾄ教､拝火教なども行なわれています｡まず一番勢力のあるものはなんといっても近代教でしょう｡生活教とも言いますがね｡｣(｢生活教｣という訳語は当たっていないかもしれません｡この原語は Quemoocha です｡cha は英吉利語の ism という意味に当たるでしょう｡quemoo の原形 quemal の訳は単に｢生きる｣というよりも｢飯を食ったり､酒を飲んだり､交合を行なったり｣する意味です｡)
｢じゃこの国にも教会だの寺院だのはあるわけなのだね?｣
｢常談を言ってはいけません｡近代教の大寺院などはこの国第一の大建築ですよ｡どうです､ちょっと見物に行っては?｣
 ある生温かい曇天の午後､ﾗｯﾌﾟは得々と僕といっしょにこの大寺院へ出かけました｡なるほどそれはﾆｺﾗｲ堂の十倍もある大建築です｡のみならずあらゆる建築様式を一つに組み上げた大建築です｡僕はこの大寺院の前に立ち､高い塔や円屋根をながめた時､なにか無気味にさえ感じました｡実際それらは天に向かって伸びた無数の触手のように見えたものです｡僕らは玄関の前にたたずんだまま､(そのまた玄関に比べてみても､どのくらい僕らは小さかったのでしょう!)しばらくこの建築よりもむしろ途方もない怪物に近い稀代の大寺院を見上げていました｡
 大寺院の内部もまた広大です｡そのｺﾘﾝﾄ風の円柱の立った中には参詣人が何人も歩いていました｡しかしそれらは僕らのように非常に小さく見えたものです｡そのうちに僕らは腰の曲がった一匹の河童に出合いました｡するとﾗｯﾌﾟはこの河童にちょっと頭を下げた上､丁寧にこう話しかけました｡
｢長老､御達者なのは何よりもです｡｣
 相手の河童もお時宜をした後､やはり丁寧に返事をしました｡
｢これはﾗｯﾌﾟさんですか? あなたも相変わらず､――(と言いかけながら､ちょっと言葉をつがなかったのはﾗｯﾌﾟの嘴の腐っているのにやっと気がついたためだったでしょう｡)――ああ､とにかく御丈夫らしいようですね｡が､きょうはどうしてまた……｣
｢きょうはこの方のお伴をしてきたのです｡この方はたぶん御承知のとおり､――｣
 それからﾗｯﾌﾟは滔々と僕のことを話しました｡どうもまたそれはこの大寺院へﾗｯﾌﾟがめったに来ないことの弁解にもなっていたらしいのです｡
｢ついてはどうかこの方の御案内を願いたいと思うのですが｡｣
 長老は大様に微笑しながら､まず僕に挨拶をし､静かに正面の祭壇を指さしました｡
｢御案内と申しても､何もお役に立つことはできません｡我々信徒の礼拝するのは正面の祭壇にある『生命の樹』です｡『生命の樹』にはごらんのとおり､金と緑との果がなっています｡あの金の果を『善の果』と言い､あの緑の果を『悪の果』と言います｡……｣
 僕はこういう説明のうちにもう退屈を感じ出しました｡それはせっかくの長老の言葉も古い比喩のように聞こえたからです｡僕はもちろん熱心に聞いている容子を装っていました｡が､時々は大寺院の内部へそっと目をやるのを忘れずにいました｡
 ｺﾘﾝﾄ風の柱､ｺﾞｼｯｸ風の穹窿､ｱﾗﾋﾞｱじみた市松模様の床､ｾｾｯｼｮﾝまがいの祈祷机､――こういうものの作っている調和は妙に野蛮な美を具えていました｡しかし僕の目をひいたのは何よりも両側の龕の中にある大理石の半身像です｡僕は何かそれらの像を見知っているように思いました｡それもまた不思議ではありません｡あの腰の曲った河童は｢生命の樹｣の説明をおわると､今度は僕やﾗｯﾌﾟといっしょに右側の龕の前へ歩み寄り､その龕の中の半身像にこういう説明を加え出しました｡
｢これは我々の聖徒のひとり､――あらゆるものに反逆した聖徒ｽﾄﾘﾝﾄﾍﾞﾘｲです｡この聖徒はさんざん苦しんだあげく､ｽｳｪﾃﾞﾝﾎﾞﾙｸﾞの哲学のために救われたように言われています｡が､実は救われなかったのです｡この聖徒はただ我々のように生活教を信じていました｡――というよりも信じるほかはなかったのでしょう｡この聖徒の我々に残した『伝説』という本を読んでごらんなさい｡この聖徒も自殺未遂者だったことは聖徒自身告白しています｡｣
 僕はちょっと憂鬱になり､次の龕へ目をやりました｡次の龕にある半身像は口髭の太い独逸人です｡
｢これはﾂｧﾗﾄｽﾄﾗの詩人ﾆｲﾁｪです｡その聖徒は聖徒自身の造った超人に救いを求めました｡が､やはり救われずに気違いになってしまったのです｡もし気違いにならなかったとすれば､あるいは聖徒の数へはいることもできなかったかもしれません｡……｣
 長老はちょっと黙った後､第三の龕の前へ案内しました｡
｢三番目にあるのはﾄﾙｽﾄｲです｡この聖徒はだれよりも苦行をしました｡それは元来貴族だったために好奇心の多い公衆に苦しみを見せることをきらったからです｡この聖徒は事実上信ぜられない基督を信じようと努力しました｡いや､信じているようにさえ公言したこともあったのです｡しかしとうとう晩年には悲壮な※(｢言+墟のつくり｣､第4水準2-88-74)つきだったことに堪えられないようになりました｡この聖徒も時々書斎の梁に恐怖を感じたのは有名です｡けれども聖徒の数にはいっているくらいですから､もちろん自殺したのではありません｡｣
 第四の龕の中の半身像は我々日本人のひとりです｡僕はこの日本人の顔を見た時､さすがに懐しさを感じました｡
｢これは国木田独歩です｡轢死する人足の心もちをはっきり知っていた詩人です｡しかしそれ以上の説明はあなたには不必要に違いありません｡では五番目の龕の中をごらんください｡――｣
｢これはﾜｸﾞﾈﾙではありませんか?｣
｢そうです｡国王の友だちだった革命家です｡聖徒ﾜｸﾞﾈﾙは晩年には食前の祈祷さえしていました｡しかしもちろん基督教よりも生活教の信徒のひとりだったのです｡ﾜｸﾞﾈﾙの残した手紙によれば､娑婆苦は何度この聖徒を死の前に駆りやったかわかりません｡｣
 僕らはもうその時には第六の龕の前に立っていました｡
｢これは聖徒ｽﾄﾘﾝﾄﾍﾞﾘｲの友だちです｡子どもの大勢ある細君の代わりに十三四のｸｲﾃｨの女をめとった商売人上がりの仏蘭西の画家です｡この聖徒は太い血管の中に水夫の血を流していました｡が､唇をごらんなさい｡砒素か何かの痕が残っています｡第七の龕の中にあるのは……もうあなたはお疲れでしょう｡ではどうかこちらへおいでください｡｣
 僕は実際疲れていましたから､ﾗｯﾌﾟといっしょに長老に従い､香の匂いのする廊下伝いにある部屋へはいりました｡そのまた小さい部屋の隅には黒いｳﾞｪﾇｽの像の下に山葡萄が一ふさ献じてあるのです｡僕はなんの装飾もない僧房を想像していただけにちょっと意外に感じました｡すると長老は僕の容子にこういう気もちを感じたとみえ､僕らに椅子を薦める前に半ば気の毒そうに説明しました｡
｢どうか我々の宗教の生活教であることを忘れずにください｡我々の神､――『生命の樹』の教えは『旺盛に生きよ』というのですから｡……ﾗｯﾌﾟさん､あなたはこのかたに我々の聖書をごらんにいれましたか?｣
｢いえ､……実はわたし自身もほとんど読んだことはないのです｡｣
 ﾗｯﾌﾟは頭の皿を掻きながら､正直にこう返事をしました｡が､長老は相変わらず静かに微笑して話しつづけました｡
｢それではおわかりなりますまい｡我々の神は一日のうちにこの世界を造りました｡(『生命の樹』は樹というものの､成しあたわないことはないのです｡)のみならず雌の河童を造りました｡すると雌の河童は退屈のあまり､雄の河童を求めました｡我々の神はこの嘆きを憐れみ､雌の河童の脳髄を取り､雄の河童を造りました｡我々の神はこの二匹の河童に『食えよ､交合せよ､旺盛に生きよ』という祝福を与えました｡……｣
 僕は長老の言葉のうちに詩人のﾄｯｸを思い出しました｡詩人のﾄｯｸは不幸にも僕のように無神論者です｡僕は河童ではありませんから､生活教を知らなかったのも無理はありません｡けれども河童の国に生まれたﾄｯｸはもちろん｢生命の樹｣を知っていたはずです｡僕はこの教えに従わなかったﾄｯｸの最後を憐れみましたから､長老の言葉をさえぎるようにﾄｯｸのことを話し出しました｡
｢ああ､あの気の毒な詩人ですね｡｣
 長老は僕の話を聞き､深い息をもらしました｡
｢我々の運命を定めるものは信仰と境遇と偶然とだけです｡(もっともあなたがたはそのほかに遺伝をお数えなさるでしょう｡)ﾄｯｸさんは不幸にも信仰をお持ちにならなかったのです｡｣
｢ﾄｯｸはあなたをうらやんでいたでしょう｡いや､僕もうらやんでいます｡ﾗｯﾌﾟ君などは年も若いし､……｣
｢僕も嘴さえちゃんとしていればあるいは楽天的だったかもしれません｡｣
 長老は僕らにこう言われると､もう一度深い息をもらしました｡しかもその目は涙ぐんだまま､じっと黒いｳﾞｪﾇｽを見つめているのです｡
｢わたしも実は､――これはわたしの秘密ですから､どうかだれにもおっしゃらずにください｡――わたしも実は我々の神を信ずるわけにいかないのです｡しかしいつかわたしの祈祷は､――｣
 ちょうど長老のこう言った時です｡突然部屋の戸があいたと思うと､大きい雌の河童が一匹､いきなり長老へ飛びかかりました｡僕らがこの雌の河童を抱きとめようとしたのはもちろんです｡が､雌の河童はとっさの間に床の上へ長老を投げ倒しました｡
｢この爺め! きょうもまたわたしの財布から一杯やる金を盗んでいったな!｣
 十分ばかりたった後､僕らは実際逃げ出さないばかりに長老夫婦をあとに残し､大寺院の玄関を下りていきました｡
｢あれではあの長老も『生命の樹』を信じないはずですね｡｣
 しばらく黙って歩いた後､ﾗｯﾌﾟは僕にこう言いました｡が､僕は返事をするよりも思わず大寺院を振り返りました｡大寺院はどんより曇った空にやはり高い塔や円屋根を無数の触手のように伸ばしています｡なにか沙漠の空に見える蜃気楼の無気味さを漂わせたまま｡……
一五
 それからかれこれ一週間の後､僕はふと医者のﾁｬｯｸに珍しい話を聞きました｡というのはあのﾄｯｸの家に幽霊の出るという話なのです｡そのころにはもう雌の河童はどこかほかへ行ってしまい､僕らの友だちの詩人の家も写真師のｽﾃｭﾃﾞｨｵに変わっていました｡なんでもﾁｬｯｸの話によれば､このｽﾃｭﾃﾞｨｵでは写真をとると､ﾄｯｸの姿もいつの間にか必ず朦朧と客の後ろに映っているとかいうことです｡もっともﾁｬｯｸは物質主義者ですから､死後の生命などを信じていません｡現にその話をした時にも悪意のある微笑を浮かべながら､｢やはり霊魂というものも物質的存在とみえますね｣などと註釈めいたことをつけ加えていました｡僕も幽霊を信じないことはﾁｬｯｸとあまり変わりません｡けれども詩人のﾄｯｸには親しみを感じていましたから､さっそく本屋の店へ駆けつけ､ﾄｯｸの幽霊に関する記事やﾄｯｸの幽霊の写真の出ている新聞や雑誌を買ってきました｡なるほどそれらの写真を見ると､どこかﾄｯｸらしい河童が一匹､老若男女の河童の後ろにぼんやりと姿を現わしていました｡しかし僕を驚かせたのはﾄｯｸの幽霊の写真よりもﾄｯｸの幽霊に関する記事､――ことにﾄｯｸの幽霊に関する心霊学協会の報告です｡僕はかなり逐語的にその報告を訳しておきましたから､下に大略を掲げることにしましょう｡ただし括弧の中にあるのは僕自身の加えた註釈なのです｡――
 詩人ﾄｯｸ君の幽霊に関する報告｡(心霊学協会雑誌第八千二百七十四号所載)
 わが心霊学協会は先般自殺したる詩人ﾄｯｸ君の旧居にして現在は写真師のｽﾃｭﾃﾞｨｵなる□□街第二百五十一号に臨時調査会を開催せり｡列席せる会員は下のごとし｡(氏名を略す｡)
 我ら十七名の会員は心霊協会会長ﾍﾟｯｸ氏とともに九月十七日午前十時三十分､我らのもっとも信頼するﾒﾃﾞｨｱﾑ､ﾎｯﾌﾟ夫人を同伴し､該ｽﾃｭﾃﾞｨｵの一室に参集せり｡ﾎｯﾌﾟ夫人は該ｽﾃｭﾃﾞｨｵにはいるや､すでに心霊的空気を感じ､全身に痙攣を催しつつ､嘔吐すること数回に及べり｡夫人の語るところによれば､こは詩人ﾄｯｸ君の強烈なる煙草を愛したる結果､その心霊的空気もまたﾆｺﾃｨﾝを含有するためなりという｡
 我ら会員はﾎｯﾌﾟ夫人とともに円卓をめぐりて黙坐したり｡夫人は三分二十五秒の後､きわめて急劇なる夢遊状態に陥り､かつ詩人ﾄｯｸ君の心霊の憑依するところとなれり｡我ら会員は年齢順に従い､夫人に憑依せるﾄｯｸ君の心霊と左のごとき問答を開始したり｡
 問 君は何ゆえに幽霊に出ずるか?
 答 死後の名声を知らんがためなり｡
 問 君――あるいは心霊諸君は死後もなお名声を欲するや?
 答 少なくとも予は欲せざるあたわず｡しかれども予の邂逅したる日本の一詩人のごときは死後の名声を軽蔑しいたり｡
 問 君はその詩人の姓名を知れりや?
 答 予は不幸にも忘れたり｡ただ彼の好んで作れる十七字詩の一章を記憶するのみ｡
 問 その詩は如何?
 答｢古池や蛙飛びこむ水の音｣｡
 問 君はその詩を佳作なりとなすや?
 答 予は必ずしも悪作なりとなさず｡ただ｢蛙｣を｢河童｣とせんか､さらに光彩陸離たるべし｡
 問 しからばその理由は如何?
 答 我ら河童はいかなる芸術にも河童を求むること痛切なればなり｡
 会長ﾍﾟｯｸ氏はこの時にあたり､我ら十七名の会員にこは心霊学協会の臨時調査会にして合評会にあらざるを注意したり｡
 問 心霊諸君の生活は如何?
 答 諸君の生活と異なることなし｡
 問 しからば君は君自身の自殺せしを後悔するや?
 答 必ずしも後悔せず｡予は心霊的生活に倦まば､さらにﾋﾟｽﾄﾙを取りて自活すべし｡
 問 自活するは容易なりや否や?
 ﾄｯｸ君の心霊はこの問に答うるにさらに問をもってしたり｡こはﾄｯｸ君を知れるものにはすこぶる自然なる応酬なるべし｡
 答 自殺するは容易なりや否や?
 問 諸君の生命は永遠なりや?
 答 我らの生命に関しては諸説紛々として信ずべからず｡幸いに我らの間にも基督教､仏教､ﾓﾊﾒｯﾄ教､拝火教等の諸宗あることを忘るるなかれ｡
 問 君自身の信ずるところは?
 答 予は常に懐疑主義者なり｡
 問 しかれども君は少なくとも心霊の存在を疑わざるべし?
 答 諸君のごとく確信するあたわず｡
 問 君の交友の多少は如何?
 答 予の交友は古今東西にわたり､三百人を下らざるべし｡その著名なるものをあぐれば､ｸﾗｲｽﾄ､ﾏｲﾝﾚﾝﾃﾞﾙ､ﾜｲﾆﾝｹﾞﾙ……
 問 君の交友は自殺者のみなりや?
 答 必ずしもしかりとせず｡自殺を弁護せるﾓﾝﾃｪﾆｭのごときは予が畏友の一人なり｡ただ予は自殺せざりし厭世主義者､――ｼｮｵﾍﾟﾝﾊｳｴﾙの輩とは交際せず｡
 問 ｼｮｵﾍﾟﾝﾊｳｴﾙは健在なりや?
 答 彼は目下心霊的厭世主義を樹立し､自活する可否を論じつつあり｡しかれどもｺﾚﾗも黴菌病なりしを知り､すこぶる安堵せるもののごとし｡
 我ら会員は相次いでﾅﾎﾟﾚｵﾝ､孔子､ﾄﾞｽﾄｴﾌｽｷｲ､ﾀﾞｱｳｨﾝ､ｸﾚｵﾊﾟﾄﾗ､釈迦､ﾃﾞﾓｽﾃﾈｽ､ﾀﾞﾝﾃ､千の利休等の心霊の消息を質問したり｡しかれどもﾄｯｸ君は不幸にも詳細に答うることをなさず､かえってﾄｯｸ君自身に関する種々のｺﾞｼｯﾌﾟを質問したり｡
 問 予の死後の名声は如何?
 答 ある批評家は｢群小詩人のひとり｣と言えり｡
 問 彼は予が詩集を贈らざりしに怨恨を含めるひとりなるべし｡予の全集は出版せられしや?
 答 君の全集は出版せられたれども､売行きはなはだ振わざるがごとし｡
 問 予の全集は三百年の後､――すなわち著作権の失われたる後､万人の購うところとなるべし｡予の同棲せる女友だちは如何?
 答 彼女は書肆ﾗｯｸ君の夫人となれり｡
 問 彼女はいまだ不幸にもﾗｯｸの義眼なるを知らざるなるべし｡予が子は如何?
 答 国立孤児院にありと聞けり｡
 ﾄｯｸ君はしばらく沈黙せる後､新たに質問を開始したり｡
 問 予が家は如何?
 答 某写真師のｽﾃｭﾃﾞｨｵとなれり｡
 問 予の机はいかになれるか?
 答 いかなれるかを知るものなし｡
 問 予は予の机の抽斗に予の秘蔵せる一束の手紙を――しかれどもこは幸いにも多忙なる諸君の関するところにあらず｡今やわが心霊界はおもむろに薄暮に沈まんとす｡予は諸君と訣別すべし｡さらば｡諸君｡さらば｡わが善良なる諸君｡
 ﾎｯﾌﾟ夫人は最後の言葉とともにふたたび急劇に覚醒したり｡我ら十七名の会員はこの問答の真なりしことを上天の神に誓って保証せんとす｡(なおまた我らの信頼するﾎｯﾌﾟ夫人に対する報酬はかつて夫人が女優たりし時の日当に従いて支弁したり｡)
一六
 僕はこういう記事を読んだ後､だんだんこの国にいることも憂鬱になってきましたから､どうか我々人間の国へ帰ることにしたいと思いました｡しかしいくら探して歩いても､僕の落ちた穴は見つかりません｡そのうちにあのﾊﾞｯｸﾞという漁夫の河童の話には､なんでもこの国の街はずれにある年をとった河童が一匹､本を読んだり､笛を吹いたり､静かに暮らしているということです｡僕はこの河童に尋ねてみれば､あるいはこの国を逃げ出す途もわかりはしないかと思いましたから､さっそく街はずれへ出かけてゆきました｡しかしそこへ行ってみると､いかにも小さい家の中に年をとった河童どころか､頭の皿も固まらない､やっと十二三の河童が一匹､悠々と笛を吹いていました｡僕はもちろん間違った家へはいったではないかと思いました｡が､念のために名をきいてみると､やはりﾊﾞｯｸﾞの教えてくれた年よりの河童に違いないのです｡
｢しかしあなたは子どものようですが……｣
｢お前さんはまだ知らないのかい? わたしはどういう運命か､母親の腹を出た時には白髪頭をしていたのだよ｡それからだんだん年が若くなり､今ではこんな子どもになったのだよ｡けれども年を勘定すれば生まれる前を六十としても､かれこれ百十五六にはなるかもしれない｡｣
 僕は部屋の中を見まわしました｡そこには僕の気のせいか､質素な椅子やﾃｴﾌﾞﾙの間に何か清らかな幸福が漂っているように見えるのです｡
｢あなたはどうもほかの河童よりもしあわせに暮らしているようですね?｣
｢さあ､それはそうかもしれない｡わたしは若い時は年よりだったし､年をとった時は若いものになっている｡従って年よりのように欲にも渇かず､若いもののように色にもおぼれない｡とにかくわたしの生涯はたといしあわせではないにもしろ､安らかだったのには違いあるまい｡｣
｢なるほどそれでは安らかでしょう｡｣
｢いや､まだそれだけでは安らかにはならない｡わたしは体も丈夫だったし､一生食うに困らぬくらいの財産を持っていたのだよ｡しかし一番しあわせだったのはやはり生まれてきた時に年よりだったことだと思っている｡｣
 僕はしばらくこの河童と自殺したﾄｯｸの話だの毎日医者に見てもらっているｹﾞｴﾙの話だのをしていました｡が､なぜか年をとった河童はあまり僕の話などに興味のないような顔をしていました｡
｢ではあなたはほかの河童のように格別生きていることに執着を持ってはいないのですね?｣
 年をとった河童は僕の顔を見ながら､静かにこう返事をしました｡
｢わたしもほかの河童のようにこの国へ生まれてくるかどうか､一応父親に尋ねられてから母親の胎内を離れたのだよ｡｣
｢しかし僕はふとした拍子に､この国へ転げ落ちてしまったのです｡どうか僕にこの国から出ていかれる路を教えてください｡｣
｢出ていかれる路は一つしかない｡｣
｢というのは?｣
｢それはお前さんのここへ来た路だ｡｣
 僕はこの答えを聞いた時になぜか身の毛がよだちました｡
｢その路があいにく見つからないのです｡｣
 年をとった河童は水々しい目にじっと僕の顔を見つめました｡それからやっと体を起こし､部屋の隅へ歩み寄ると､天井からそこに下がっていた一本の綱を引きました｡すると今まで気のつかなかった天窓が一つ開きました｡そのまた円い天窓の外には松や檜が枝を張った向こうに大空が青あおと晴れ渡っています｡いや､大きい鏃に似た槍ヶ岳の峯もそびえています｡僕は飛行機を見た子どものように実際飛び上がって喜びました｡
｢さあ､あすこから出ていくがいい｡｣
 年をとった河童はこう言いながら､さっきの綱を指さしました｡今まで僕の綱と思っていたのは実は綱梯子にできていたのです｡
｢ではあすこから出さしてもらいます｡｣
｢ただわたしは前もって言うがね｡出ていって後悔しないように｡｣
｢大丈夫です｡僕は後悔などはしません｡｣
 僕はこう返事をするが早いか､もう綱梯子をよじ登っていました｡年をとった河童の頭の皿をはるか下にながめながら｡
一七
 僕は河童の国から帰ってきた後､しばらくは我々人間の皮膚の匂いに閉口しました｡我々人間に比べれば､河童は実に清潔なものです｡のみならず我々人間の頭は河童ばかり見ていた僕にはいかにも気味の悪いものに見えました｡これはあるいはあなたにはおわかりにならないかもしれません｡しかし目や口はともかくも､この鼻というものは妙に恐ろしい気を起こさせるものです｡僕はもちろんできるだけ､だれにも会わない算段をしました｡が､我々人間にもいつか次第に慣れ出したとみえ､半年ばかりたつうちにどこへでも出るようになりました｡ただそれでも困ったことは何か話をしているうちにうっかり河童の国の言葉を口に出してしまうことです｡
｢君はあしたは家にいるかね?｣
｢Qua｣
｢なんだって?｣
｢いや､いるということだよ｡｣
 だいたいこういう調子だったものです｡
 しかし河童の国から帰ってきた後､ちょうど一年ほどたった時､僕はある事業の失敗したために……(S博士は彼がこう言った時､｢その話はおよしなさい｣と注意をした｡なんでも博士の話によれば､彼はこの話をするたびに看護人の手にもおえないくらい､乱暴になるとかいうことである｡)
 ではその話はやめましょう｡しかしある事業の失敗したために僕はまた河童の国へ帰りたいと思い出しました｡そうです｡｢行きたい｣のではありません｡｢帰りたい｣と思い出したのです｡河童の国は当時の僕には故郷のように感ぜられましたから｡
 僕はそっと家を脱け出し､中央線の汽車へ乗ろうとしました｡そこをあいにく巡査につかまり､とうとう病院へ入れられたのです｡僕はこの病院へはいった当座も河童の国のことを想いつづけました｡医者のﾁｬｯｸはどうしているでしょう? 哲学者のﾏｯｸﾞも相変わらず七色の色硝子のﾗﾝﾀｱﾝの下に何か考えているかもしれません｡ことに僕の親友だった嘴の腐った学生のﾗｯﾌﾟは､――あるきょうのように曇った午後です｡こんな追憶にふけっていた僕は思わず声をあげようとしました｡それはいつの間にはいってきたか､ﾊﾞｯｸﾞという漁夫の河童が一匹､僕の前にたたずみながら､何度も頭を下げていたからです｡僕は心をとり直した後､――泣いたか笑ったかも覚えていません｡が､とにかく久しぶりに河童の国の言葉を使うことに感動していたことはたしかです｡
｢おい､ﾊﾞｯｸﾞ､どうして来た?｣
｢へい､お見舞いに上がったのです｡なんでも御病気だとかいうことですから｡｣
｢どうしてそんなことを知っている?｣
｢ﾗﾃﾞｨｵのﾆｳｽで知ったのです｡｣
 ﾊﾞｯｸﾞは得意そうに笑っているのです｡
｢それにしてもよく来られたね?｣
｢なに､造作はありません｡東京の川や掘割りは河童には往来も同様ですから｡｣
 僕は河童も蛙のように水陸両棲の動物だったことに今さらのように気がつきました｡
｢しかしこの辺には川はないがね｡｣
｢いえ､こちらへ上がったのは水道の鉄管を抜けてきたのです｡それからちょっと消火栓をあけて……｣
｢消火栓をあけて?｣
｢旦那はお忘れなすったのですか? 河童にも機械屋のいるということを｡｣
 それから僕は二三日ごとにいろいろの河童の訪問を受けました｡僕の病はS博士によれば早発性痴呆症ということです｡しかしあの医者のﾁｬｯｸは(これははなはだあなたにも失礼に当たるのに違いありません｡)僕は早発性痴呆症患者ではない､早発性痴呆症患者はS博士をはじめ､あなたがた自身だと言っていました｡医者のﾁｬｯｸも来るくらいですから､学生のﾗｯﾌﾟや哲学者のﾏｯｸﾞの見舞いにきたことはもちろんです｡が､あの漁夫のﾊﾞｯｸﾞのほかに昼間はだれも尋ねてきません｡ことに二三匹いっしょに来るのは夜､――それも月のある夜です｡僕はゆうべも月明りの中に硝子会社の社長のｹﾞｴﾙや哲学者のﾏｯｸﾞと話をしました｡のみならず音楽家のｸﾗﾊﾞｯｸにもｳﾞｧｲｵﾘﾝを一曲弾いてもらいました｡そら､向こうの机の上に黒百合の花束がのっているでしょう? あれもゆうべｸﾗﾊﾞｯｸが土産に持ってきてくれたものです｡……
 (僕は後ろを振り返ってみた｡が､もちろん机の上には花束も何ものっていなかった｡)
 それからこの本も哲学者のﾏｯｸﾞがわざわざ持ってきてくれたものです｡ちょっと最初の詩を読んでごらんなさい｡いや､あなたは河童の国の言葉を御存知になるはずはありません｡では代わりに読んでみましょう｡これは近ごろ出版になったﾄｯｸの全集の一冊です｡――
 (彼は古い電話帳をひろげ､こういう詩をおお声に読みはじめた｡)
――椰子の花や竹の中に
  仏陀はとうに眠っている｡
  路ばたに枯れた無花果といっしょに
  基督ももう死んだらしい｡
  しかし我々は休まなければならぬ
  たとい芝居の背景の前にも｡
  (そのまた背景の裏を見れば､継ぎはぎだらけのｶﾝｳﾞｧｽばかりだ?)――
 けれども僕はこの詩人のように厭世的ではありません｡河童たちの時々来てくれる限りは､――ああ､このことは忘れていました｡あなたは僕の友だちだった裁判官のﾍﾟｯﾌﾟを覚えているでしょう｡あの河童は職を失った後､ほんとうに発狂してしまいました｡なんでも今は河童の国の精神病院にいるということです｡僕はS博士さえ承知してくれれば､見舞いにいってやりたいのですがね……｡"""
    },
    {
    "@search.action": "upload",
    "BookId": "55",
    "BookName": "芋粥",
    "BookAuthor": "芥川龍之介",
    "BookGenre": "短編小説",
    "Release_Date": "1916年9月",
    "Rating": "3.38",
    "Text": """元慶の末か､仁和の始にあつた話であらう｡どちらにしても時代はさして､この話に大事な役を､勤めてゐない｡読者は唯､平安朝と云ふ､遠い昔が背景になつてゐると云ふ事を､知つてさへゐてくれれば､よいのである｡――その頃､摂政藤原基経に仕へてゐる侍の中に､某と云ふ五位があつた｡
 これも､某と書かずに､何の誰と､ちやんと姓名を明にしたいのであるが､生憎旧記には､それが伝はつてゐない｡恐らくは､実際､伝はる資格がない程､平凡な男だつたのであらう｡一体旧記の著者などと云ふ者は､平凡な人間や話に､余り興味を持たなかつたらしい｡この点で､彼等と､日本の自然派の作家とは､大分ちがふ｡王朝時代の小説家は､存外､閑人でない｡――兎に角､摂政藤原基経に仕へてゐる侍の中に､某と云ふ五位があつた｡これが､この話の主人公である｡
 五位は､風采の甚揚らない男であつた｡第一背が低い｡それから赤鼻で､眼尻が下つてゐる｡口髭は勿論薄い｡頬が､こけてゐるから､頤が､人並はづれて､細く見える｡唇は――一々､数へ立ててゐれば､際限はない｡我五位の外貌はそれ程､非凡に､だらしなく､出来上つてゐたのである｡
 この男が､何時､どうして､基経に仕へるやうになつたのか､それは誰も知つてゐない｡が､余程以前から､同じやうな色の褪めた水干に､同じやうな萎々した烏帽子をかけて､同じやうな役目を､飽きずに､毎日､繰返してゐる事だけは､確である｡その結果であらう､今では､誰が見ても､この男に若い時があつたとは思はれない｡(五位は四十を越してゐた｡)その代り､生れた時から､あの通り寒むさうな赤鼻と､形ばかりの口髭とを､朱雀大路の衢風に､吹かせてゐたと云ふ気がする｡上は主人の基経から､下は牛飼の童児まで､無意識ながら､悉さう信じて疑ふ者がない｡
 かう云ふ風采を具へた男が､周囲から受ける待遇は､恐らく書くまでもないことであらう｡侍所にゐる連中は､五位に対して､殆ど蠅程の注意も払はない｡有位無位､併せて二十人に近い下役さへ､彼の出入りには､不思議な位､冷淡を極めてゐる｡五位が何か云ひつけても､決して彼等同志の雑談をやめた事はない｡彼等にとつては､空気の存在が見えないやうに､五位の存在も､眼を遮らないのであらう｡下役でさへさうだとすれば､別当とか､侍所の司とか云ふ上役たちが頭から彼を相手にしないのは､寧ろ自然の数である｡彼等は､五位に対すると､殆ど､子供らしい無意味な悪意を､冷然とした表情の後に隠して､何を云ふのでも､手真似だけで用を足した｡人間に､言語があるのは､偶然ではない｡従つて､彼等も手真似では用を弁じない事が､時々ある｡が､彼等は､それを全然五位の悟性に､欠陥があるからだと､思つてゐるらしい｡そこで彼等は用が足りないと､この男の歪んだ揉烏帽子の先から､切れかかつた藁草履の尻まで､万遍なく見上げたり､見下したりして､それから､鼻で哂ひながら､急に後を向いてしまふ｡それでも､五位は､腹を立てた事がない｡彼は､一切の不正を､不正として感じない程､意気地のない､臆病な人間だつたのである｡
 所が､同僚の侍たちになると､進んで､彼を飜弄しようとした｡年かさの同僚が､彼れの振はない風采を材料にして､古い洒落を聞かせようとする如く､年下の同僚も､亦それを機会にして､所謂興言利口の練習をしようとしたからである｡彼等は､この五位の面前で､その鼻と口髭と､烏帽子と水干とを､品隲して飽きる事を知らなかつた｡そればかりではない｡彼が五六年前に別れたうけ唇の女房と､その女房と関係があつたと云ふ酒のみの法師とも､屡彼等の話題になつた｡その上､どうかすると､彼等は甚､性質の悪い悪戯さへする｡それを今一々､列記する事は出来ない｡が､彼の篠枝の酒を飲んで､後へ尿を入れて置いたと云ふ事を書けば､その外は凡､想像される事だらうと思ふ｡
 しかし､五位はこれらの揶揄に対して､全然無感覚であつた｡少くもわき眼には､無感覚であるらしく思はれた｡彼は何を云はれても､顔の色さへ変へた事がない｡黙つて例の薄い口髭を撫でながら､するだけの事をしてすましてゐる｡唯､同僚の悪戯が､嵩じすぎて､髷に紙切れをつけたり､太刀の鞘に草履を結びつけたりすると､彼は笑ふのか､泣くのか､わからないやうな笑顔をして､｢いけぬのう､お身たちは｡｣と云ふ｡その顔を見､その声を聞いた者は､誰でも一時或いぢらしさに打たれてしまふ｡(彼等にいぢめられるのは､一人､この赤鼻の五位だけではない､彼等の知らない誰かが――多数の誰かが､彼の顔と声とを借りて､彼等の無情を責めてゐる｡)――さう云ふ気が､朧げながら､彼等の心に､一瞬の間､しみこんで来るからである｡唯その時の心もちを､何時までも持続ける者は甚少い｡その少い中の一人に､或無位の侍があつた｡これは丹波の国から来た男で､まだ柔かい口髭が､やつと鼻の下に､生えかかつた位の青年である｡勿論､この男も始めは皆と一しよに､何の理由もなく､赤鼻の五位を軽蔑した｡所が､或日何かの折に､｢いけぬのう､お身たちは｣と云ふ声を聞いてからは､どうしても､それが頭を離れない｡それ以来､この男の眼にだけは､五位が全く別人として､映るやうになつた｡栄養の不足した､血色の悪い､間のぬけた五位の顔にも､世間の迫害にべそを掻いた､｢人間｣が覗いてゐるからである｡この無位の侍には､五位の事を考へる度に､世の中のすべてが急に本来の下等さを露すやうに思はれた｡さうしてそれと同時に霜げた赤鼻と数へる程の口髭とが何となく一味の慰安を自分の心に伝へてくれるやうに思はれた｡……
 しかし､それは､唯この男一人に､限つた事である｡かう云ふ例外を除けば､五位は､依然として周囲の軽蔑の中に､犬のやうな生活を続けて行かなければならなかつた｡第一彼には着物らしい着物が一つもない｡青鈍の水干と､同じ色の指貫とが一つづつあるのが､今ではそれが上白んで､藍とも紺とも､つかないやうな色に､なつてゐる｡水干はそれでも､肩が少し落ちて､丸組の緒や菊綴の色が怪しくなつてゐるだけだが､指貫になると､裾のあたりのいたみ方が一通りでない｡その指貫の中から､下の袴もはかない､細い足が出てゐるのを見ると､口の悪い同僚でなくとも､痩公卿の車を牽いてゐる､痩牛の歩みを見るやうな､みすぼらしい心もちがする｡それに佩いてゐる太刀も､頗る覚束ない物で､柄の金具も如何はしければ､黒鞘の塗も剥げかかつてゐる｡これが例の赤鼻で､だらしなく草履をひきずりながら､唯でさへ猫背なのを､一層寒空の下に背ぐくまつて､もの欲しさうに､左右を眺め眺め､きざみ足に歩くのだから､通りがかりの物売りまで莫迦にするのも､無理はない｡現に､かう云ふ事さへあつた｡……
 或る日､五位が三条坊門を神泉苑の方へ行く所で､子供が六七人､路ばたに集つて､何かしてゐるのを見た事がある｡｢こまつぶり｣でも､廻してゐるのかと思つて､後ろから覗いて見ると､何処かから迷つて来た､尨犬の首へ繩をつけて､打つたり殴いたりしてゐるのであつた｡臆病な五位は､これまで何かに同情を寄せる事があつても､あたりへ気を兼ねて､まだ一度もそれを行為に現はしたことがない｡が､この時だけは相手が子供だと云ふので､幾分か勇気が出た｡そこで出来るだけ､笑顔をつくりながら､年かさらしい子供の肩を叩いて､｢もう､堪忍してやりなされ｡犬も打たれれば､痛いでのう｣と声をかけた｡すると､その子供はふりかへりながら､上眼を使つて､蔑すむやうに､ぢろぢろ五位の姿を見た｡云はば侍所の別当が用の通じない時に､この男を見るやうな顔をして､見たのである｡｢いらぬ世話はやかれたうもない｡｣その子供は一足下りながら､高慢な唇を反らせて､かう云つた｡｢何ぢや､この鼻赤めが｡｣五位はこの語が自分の顔を打つたやうに感じた｡が､それは悪態をつかれて､腹が立つたからでは毛頭ない｡云はなくともいい事を云つて､恥をかいた自分が､情なくなつたからである｡彼は､きまりが悪いのを苦しい笑顔に隠しながら､黙つて､又､神泉苑の方へ歩き出した｡後では､子供が､六七人､肩を寄せて､｢べつかつかう｣をしたり､舌を出したりしてゐる｡勿論彼はそんな事を知らない｡知つてゐたにしても､それが､この意気地のない五位にとつて､何であらう｡……
 では､この話の主人公は､唯､軽蔑される為にのみ生れて来た人間で､別に何の希望も持つてゐないかと云ふと､さうでもない｡五位は五六年前から芋粥と云ふ物に､異常な執着を持つてゐる｡芋粥とは山の芋を中に切込んで､それを甘葛の汁で煮た､粥の事を云ふのである｡当時はこれが､無上の佳味として､上は万乗の君の食膳にさへ､上せられた｡従つて､吾五位の如き人間の口へは､年に一度､臨時の客の折にしか､はいらない｡その時でさへ､飲めるのは僅に喉を沾すに足る程の少量である｡そこで芋粥を飽きる程飲んで見たいと云ふ事が､久しい前から､彼の唯一の欲望になつてゐた｡勿論､彼は､それを誰にも話した事がない｡いや彼自身さへそれが､彼の一生を貫いてゐる欲望だとは､明白に意識しなかつた事であらう｡が事実は彼がその為に､生きてゐると云つても､差支ない程であつた｡――人間は､時として､充されるか充されないか､わからない欲望の為に､一生を捧げてしまふ｡その愚を哂ふ者は､畢竟､人生に対する路傍の人に過ぎない｡
 しかし､五位が夢想してゐた､｢芋粥に飽かむ｣事は､存外容易に事実となつて現れた｡その始終を書かうと云ふのが､芋粥の話の目的なのである｡
       ―――――――――――――――――
 或年の正月二日､基経の第に､所謂臨時の客があつた時の事である｡(臨時の客は二宮の大饗と同日に摂政関白家が､大臣以下の上達部を招いて催す饗宴で､大饗と別に変りがない｡)五位も､外の侍たちにまじつて､その残肴の相伴をした｡当時はまだ､取食みの習慣がなくて､残肴は､その家の侍が一堂に集まつて､食ふ事になつてゐたからである｡尤も､大饗に等しいと云つても昔の事だから､品数の多い割りに碌な物はない､餅､伏菟､蒸鮑､干鳥､宇治の氷魚､近江の鮒､鯛の楚割､鮭の内子､焼蛸､大海老､大柑子､小柑子､橘､串柿などの類である｡唯､その中に､例の芋粥があつた｡五位は毎年､この芋粥を楽しみにしてゐる｡が､何時も人数が多いので､自分が飲めるのは､いくらもない｡それが今年は､特に､少かつた｡さうして気のせゐか､何時もより､余程味が好い｡そこで､彼は飲んでしまつた後の椀をしげしげと眺めながら､うすい口髭についてゐる滴を､掌で拭いて誰に云ふともなく､｢何時になつたら､これに飽ける事かのう｣と､かう云つた｡
｢大夫殿は､芋粥に飽かれた事がないさうな｡｣
 五位の語が完らない中に､誰かが､嘲笑つた｡錆のある､鷹揚な､武人らしい声である｡五位は､猫背の首を挙げて､臆病らしく､その人の方を見た｡声の主は､その頃同じ基経の恪勤になつてゐた､民部卿時長の子藤原利仁である｡肩幅の広い､身長の群を抜いた逞しい大男で､これは､ﾕﾃﾞｸﾞﾘを噛みながら､黒酒の杯を重ねてゐた｡もう大分酔がまはつてゐるらしい｡
｢お気の毒な事ぢやの｡｣利仁は､五位が顔を挙げたのを見ると､軽蔑と憐憫とを一つにしたやうな声で､語を継いだ｡｢お望みなら､利仁がお飽かせ申さう｡｣
 始終､いぢめられてゐる犬は､たまに肉を貰つても容易によりつかない｡五位は､例の笑ふのか､泣くのか､わからないやうな笑顔をして､利仁の顔と､空の椀とを等分に見比べてゐた｡
｢おいやかな｡｣
｢……｣
｢どうぢや｡｣
｢……｣
 五位は､その中に､衆人の視線が､自分の上に､集まつてゐるのを感じ出した｡答へ方一つで､又､一同の嘲弄を､受けなければならない｡或は､どう答へても､結局､莫迦にされさうな気さへする｡彼は躊躇した｡もし､その時に､相手が､少し面倒臭そうな声で､｢おいやなら､たつてとは申すまい｣と云はなかつたなら､五位は､何時までも､椀と利仁とを､見比べてゐた事であらう｡
 彼は､それを聞くと､慌しく答へた｡
｢いや……忝うござる｡｣
 この問答を聞いてゐた者は､皆､一時に､失笑した｡｢いや……忝うござる｡｣――かう云つて､五位の答を､真似る者さへある｡所謂､橙黄橘紅を盛つた窪坏や高坏の上に多くの揉烏帽子や立烏帽子が､笑声と共に一しきり､波のやうに動いた｡中でも､最､大きな声で､機嫌よく､笑つたのは､利仁自身である｡
｢では､その中に､御誘ひ申さう｡｣さう云ひながら､彼は､ちよいと顔をしかめた｡こみ上げて来る笑と今飲んだ酒とが､喉で一つになつたからである｡｢……しかと､よろしいな｡｣
｢忝うござる｡｣
 五位は赤くなつて､吃りながら､又､前の答を繰返した｡一同が今度も､笑つたのは､云ふまでもない｡それが云はせたさに､わざわざ念を押した当の利仁に至つては､前よりも一層可笑しさうに広い肩をゆすつて､哄笑した｡この朔北の野人は､生活の方法を二つしか心得てゐない｡一つは酒を飲む事で､他の一つは笑ふ事である｡
 しかし幸に談話の中心は､程なく､この二人を離れてしまつた｡これは事によると､外の連中が､たとひ嘲弄にしろ､一同の注意をこの赤鼻の五位に集中させるのが､不快だつたからかも知れない｡兎に角､談柄はそれからそれへと移つて､酒も肴も残少になつた時分には､某と云ふ侍学生が､行縢の片皮へ､両足を入れて馬に乗らうとした話が､一座の興味を集めてゐた｡が､五位だけは､まるで外の話が聞えないらしい｡恐らく芋粥の二字が､彼のすべての思量を支配してゐるからであらう｡前に雉子の炙いたのがあつても､箸をつけない｡黒酒の杯があつても､口を触れない｡彼は､唯､両手を膝の上に置いて､見合ひをする娘のやうに霜に犯されかかつた鬢の辺まで､初心らしく上気しながら､何時までも空になつた黒塗の椀を見つめて､多愛もなく､微笑してゐるのである｡……
       ―――――――――――――――――
 それから､四五日たつた日の午前､加茂川の河原に沿つて､粟田口へ通ふ街道を､静に馬を進めてゆく二人の男があつた｡一人は濃い縹の狩衣に同じ色の袴をして､打出の太刀を佩いた｢鬚黒く鬢ぐきよき｣男である｡もう一人は､みすぼらしい青鈍の水干に､薄綿の衣を二つばかり重ねて着た､四十恰好の侍で､これは､帯のむすび方のだらしのない容子と云ひ､赤鼻でしかも穴のあたりが､洟にぬれてゐる容子と云ひ､身のまはり万端のみすぼらしい事夥しい｡尤も､馬は二人とも､前のは月毛､後のは蘆毛の三歳駒で､道をゆく物売りや侍も､振向いて見る程の駿足である｡その後から又二人､馬の歩みに遅れまいとして随いて行くのは､調度掛と舎人とに相違ない｡――これが､利仁と五位との一行である事は､わざわざ､ここに断るまでもない話であらう｡
 冬とは云ひながら､物静に晴れた日で､白けた河原の石の間､潺湲たる水の辺に立枯れてゐる蓬の葉を､ゆする程の風もない｡川に臨んだ背の低い柳は､葉のない枝に飴の如く滑かな日の光りをうけて､梢にゐる鶺鴒の尾を動かすのさへ､鮮かに､それと､影を街道に落してゐる｡東山の暗い緑の上に､霜に焦げた天鵞絨のやうな肩を､丸々と出してゐるのは､大方､比叡の山であらう｡二人はその中に鞍の螺鈿を､まばゆく日にきらめかせながら鞭をも加へず悠々と､粟田口を指して行くのである｡
｢どこでござるかな､手前をつれて行つて､やらうと仰せられるのは｡｣五位が馴れない手に手綱をかいくりながら､云つた｡
｢すぐ､そこぢや｡お案じになる程遠くはない｡｣
｢すると､粟田口辺でござるかな｡｣
｢まづ､さう思はれたがよろしからう｡｣
 利仁は今朝五位を誘ふのに､東山の近くに湯の湧いてゐる所があるから､そこへ行かうと云つて出て来たのである｡赤鼻の五位は､それを真にうけた｡久しく湯にはいらないので､体中がこの間からむづ痒い｡芋粥の馳走になつた上に､入湯が出来れば､願つてもない仕合せである｡かう思つて､予め利仁が牽かせて来た､蘆毛の馬に跨つた｡所が､轡を並べて此処まで来て見ると､どうも利仁はこの近所へ来るつもりではないらしい｡現に､さうかうしてゐる中に､粟田口は通りすぎた｡
｢粟田口では､ござらぬのう｡｣
｢いかにも､もそつと､あなたでな｡｣
 利仁は､微笑を含みながら､わざと､五位の顔を見ないやうにして､静に馬を歩ませてゐる｡両側の人家は､次第に稀になつて､今は､広々とした冬田の上に､餌をあさる鴉が見えるばかり､山の陰に消残つて､雪の色も仄に青く煙つてゐる｡晴れながら､とげとげしい櫨の梢が､眼に痛く空を刺してゐるのさへ､何となく肌寒い｡
｢では､山科辺ででもござるかな｡｣
｢山科は､これぢや｡もそつと､さきでござるよ｡｣
 成程､さう云ふ中に､山科も通りすぎた｡それ所ではない｡何かとする中に､関山も後にして､彼是､午少しすぎた時分には､とうとう三井寺の前へ来た｡三井寺には､利仁の懇意にしてゐる僧がある｡二人はその僧を訪ねて､午餐の馳走になつた｡それがすむと､又､馬に乗つて､途を急ぐ｡行手は今まで来た路に比べると遙に人煙が少ない｡殊に当時は盗賊が四方に横行した､物騒な時代である｡――五位は猫背を一層低くしながら､利仁の顔を見上げるやうにして訊ねた｡
｢まだ､さきでござるのう｡｣
 利仁は微笑した｡悪戯をして､それを見つけられさうになつた子供が､年長者に向つてするやうな微笑である｡鼻の先へよせた皺と､眼尻にたたへた筋肉のたるみとが､笑つてしまはうか､しまふまいかとためらつてゐるらしい｡さうして､とうとう､かう云つた｡
｢実はな､敦賀まで､お連れ申さうと思うたのぢや｡｣笑ひながら､利仁は鞭を挙げて遠くの空を指さした｡その鞭の下には､ﾃｷﾚｷとして､午後の日を受けた近江の湖が光つてゐる｡
 五位は､狼狽した｡
｢敦賀と申すと､あの越前の敦賀でござるかな｡あの越前の――｣
 利仁が､敦賀の人､藤原有仁の女婿になつてから､多くは敦賀に住んでゐると云ふ事も､日頃から聞いてゐない事はない｡が､その敦賀まで自分をつれて行く気だらうとは､今の今まで思はなかつた｡第一､幾多の山河を隔ててゐる越前の国へ､この通り､僅二人の伴人をつれただけで､どうして無事に行かれよう｡ましてこの頃は､往来の旅人が､盗賊の為に殺されたと云ふ噂さへ､諸方にある｡――五位は歎願するやうに､利仁の顔を見た｡
｢それは又､滅相な､東山ぢやと心得れば､山科｡山科ぢやと心得れば､三井寺｡揚句が越前の敦賀とは､一体どうしたと云ふ事でござる｡始めから､さう仰せられうなら､下人共なりと､召つれようものを｡――敦賀とは､滅相な｡｣
 五位は､殆どべそを掻かないばかりになつて､呟いた｡もし｢芋粥に飽かむ｣事が､彼の勇気を鼓舞しなかつたとしたら､彼は恐らく､そこから別れて､京都へ独り帰つて来た事であらう｡
｢利仁が一人居るのは､千人ともお思ひなされ｡路次の心配は､御無用ぢや｡｣
 五位の狼狽するのを見ると､利仁は､少し眉を顰めながら､嘲笑つた｡さうして調度掛を呼寄せて､持たせて来たﾂﾎﾞﾔﾅｸﾞﾋを背に負ふと､やはり､その手から､黒漆の真弓をうけ取つて､それを鞍上に横へながら､先に立つて､馬を進めた｡かうなる以上､意気地のない五位は､利仁の意志に盲従するより外に仕方がない｡それで､彼は心細さうに､荒涼とした周囲の原野を眺めながら､うろ覚えの観音経を口の中に念じ念じ､例の赤鼻を鞍の前輪にすりつけるやうにして､覚束ない馬の歩みを､不相変とぼとぼと進めて行つた｡
 馬蹄の反響する野は､茫々たる黄茅に蔽はれて､その所々にある行潦も､つめたく､青空を映したまま､この冬の午後を､何時かそれなり凍つてしまふかと疑はれる｡その涯には､一帯の山脈が､日に背いてゐるせゐか､かがやく可き残雪の光もなく､紫がかつた暗い色を､長々となすつてゐるが､それさへ蕭条たる幾叢の枯薄に遮られて､二人の従者の眼には､はいらない事が多い｡――すると､利仁が､突然､五位の方をふりむいて､声をかけた｡
｢あれに､よい使者が参つた｡敦賀への言づけを申さう｡｣
 五位は利仁の云ふ意味が､よくわからないので､怖々ながら､その弓で指さす方を､眺めて見た｡元より人の姿が見えるやうな所ではない｡唯､野葡萄か何かの蔓が､灌木の一むらにからみついてゐる中を､一疋の狐が､暖かな毛の色を､傾きかけた日に曝しながら､のそりのそり歩いて行く｡――と思ふ中に､狐は､慌ただしく身を跳らせて､一散に､どこともなく走り出した｡利仁が急に､鞭を鳴らせて､その方へ馬を飛ばし始めたからである｡五位も､われを忘れて､利仁の後を､逐つた｡従者も勿論､遅れてはゐられない｡しばらくは､石を蹴る馬蹄の音が､戞々として､曠野の静けさを破つてゐたが､やがて利仁が､馬を止めたのを見ると､何時､捕へたのか､もう狐の後足を掴んで､倒に､鞍の側へ､ぶら下げてゐる｡狐が､走れなくなるまで､追ひつめた所で､それを馬の下に敷いて､手取りにしたものであらう｡五位は､うすい髭にたまる汗を､慌しく拭きながら､漸､その傍へ馬を乗りつけた｡
｢これ､狐､よう聞けよ｡｣利仁は､狐を高く眼の前へつるし上げながら､わざと物々しい声を出してかう云つた｡｢其方､今夜の中に､敦賀の利仁が館へ参つて､かう申せ｡『利仁は､唯今俄に客人を具して下らうとする所ぢや｡明日､巳時頃､高島の辺まで､男たちを迎ひに遣はし､それに､鞍置馬二疋､牽かせて参れ｡』よいか忘れるなよ｡｣
 云ひ畢ると共に､利仁は､一ふり振つて狐を､遠くの叢の中へ､抛り出した｡
｢いや､走るわ｡走るわ｡｣
 やつと､追ひついた二人の従者は､逃げてゆく狐の行方を眺めながら､手を拍つて囃し立てた｡落葉のやうな色をしたその獣の背は､夕日の中を､まつしぐらに､木の根石くれの嫌ひなく､何処までも､走つて行く｡それが一行の立つてゐる所から､手にとるやうによく見えた｡狐を追つてゐる中に､何時か彼等は､曠野が緩い斜面を作つて､水の涸れた川床と一つになる､その丁度上の所へ､出てゐたからである｡
｢広量の御使でござるのう｡｣
 五位は､ﾅｲｲｳﾞな尊敬と讃嘆とを洩らしながら､この狐さへ頤使する野育ちの武人の顔を､今更のやうに､仰いで見た｡自分と利仁との間に､どれ程の懸隔があるか､そんな事は､考へる暇がない｡唯､利仁の意志に､支配される範囲が広いだけに､その意志の中に包容される自分の意志も､それだけ自由が利くやうになつた事を､心強く感じるだけである｡――阿諛は､恐らく､かう云ふ時に､最自然に生れて来るものであらう｡読者は､今後､赤鼻の五位の態度に､幇間のやうな何物かを見出しても､それだけで妄にこの男の人格を､疑ふ可きではない｡
 抛り出された狐は､なぞへの斜面を､転げるやうにして､駈け下りると､水の無い河床の石の間を､器用に､ぴよいぴよい､飛び越えて､今度は､向うの斜面へ､勢よく､すぢかひに駈け上つた｡駈け上りながら､ふりかへつて見ると､自分を手捕りにした侍の一行は､まだ遠い傾斜の上に馬を並べて立つてゐる｡それが皆､指を揃へた程に､小さく見えた｡殊に入日を浴びた､月毛と蘆毛とが､霜を含んだ空気の中に､描いたよりもくつきりと､浮き上つてゐる｡
 狐は､頭をめぐらすと､又枯薄の中を､風のやうに走り出した｡
       ―――――――――――――――――
 一行は､予定通り翌日の巳時ばかりに､高島の辺へ来た｡此処は琵琶湖に臨んだ､ささやかな部落で､昨日に似ず､どんよりと曇つた空の下に､幾戸の藁屋が､疎にちらばつてゐるばかり､岸に生えた松の樹の間には､灰色の漣※(｢さんずい+猗｣､第3水準1-87-6)をよせる湖の水面が､磨くのを忘れた鏡のやうに､さむざむと開けてゐる｡――此処まで来ると利仁が､五位を顧みて云つた｡
｢あれを御覧じろ｡男どもが､迎ひに参つたげでござる｡｣
 見ると､成程､二疋の鞍置馬を牽いた､二三十人の男たちが､馬に跨がつたのもあり徒歩のもあり､皆水干の袖を寒風に翻へして､湖の岸､松の間を､一行の方へ急いで来る｡やがてこれが､間近くなつたと思ふと､馬に乗つてゐた連中は､慌ただしく鞍を下り､徒歩の連中は､路傍に蹲踞して､いづれも恭々しく､利仁の来るのを､待ちうけた｡
｢やはり､あの狐が､使者を勤めたと見えますのう｡｣
｢生得､変化ある獣ぢやて､あの位の用を勤めるのは､何でもござらぬ｡｣
 五位と利仁とが､こんな話をしてゐる中に､一行は､郎等たちの待つてゐる所へ来た｡｢大儀ぢや｡｣と､利仁が声をかける｡蹲踞してゐた連中が､忙しく立つて､二人の馬の口を取る｡急に､すべてが陽気になつた｡
｢夜前､稀有な事が､ございましてな｡｣
 二人が､馬から下りて､敷皮の上へ､腰を下すか下さない中に､檜皮色の水干を着た､白髪の郎等が､利仁の前へ来て､かう云つた｡｢何ぢや｡｣利仁は､郎等たちの持つて来た篠枝や破籠を､五位にも勧めながら､鷹揚に問ひかけた｡
｢さればでございまする｡夜前､戌時ばかりに､奥方が俄に､人心地をお失ひなされましてな｡『おのれは､阪本の狐ぢや｡今日､殿の仰せられた事を､言伝てせうほどに､近う寄つて､よう聞きやれ｡』と､かう仰有るのでございまする｡さて､一同がお前に参りますると､奥方の仰せられまするには､『殿は唯今俄に客人を具して､下られようとする所ぢや｡明日巳時頃､高島の辺まで､男どもを迎ひに遺はし､それに鞍置馬二疋牽かせて参れ｡』と､かう御意遊ばすのでございまする｡｣
｢それは､又､稀有な事でござるのう｡｣五位は利仁の顔と､郎等の顔とを､仔細らしく見比べながら､両方に満足を与へるやうな､相槌を打つた｡
｢それも唯､仰せられるのではございませぬ｡さも､恐ろしさうに､わなわなとお震へになりましてな､『遅れまいぞ｡遅れれば､おのれが､殿の御勘当をうけねばならぬ｡』と､しつきりなしに､お泣きになるのでございまする｡｣
｢して､それから､如何した｡｣
｢それから､多愛なく､お休みになりましてな｡手前共の出て参りまする時にも､まだ､お眼覚にはならぬやうで､ございました｡｣
｢如何でござるな｡｣郎等の話を聞き完ると､利仁は五位を見て､得意らしく云つた｡｢利仁には､獣も使はれ申すわ｡｣
｢何とも驚き入る外は､ござらぬのう｡｣五位は､赤鼻を掻きながら､ちよいと､頭を下げて､それから､わざとらしく､呆れたやうに､口を開いて見せた｡口髭には､今飲んだ酒が､滴になつて､くつついてゐる｡
       ―――――――――――――――――
その日の夜の事である｡五位は､利仁の館の一間に､切燈台の灯を眺めるともなく､眺めながら､寝つかれない長の夜をまぢまぢして､明してゐた｡すると､夕方､此処へ着くまでに､利仁や利仁の従者と､談笑しながら､越えて来た松山､小川､枯野､或は､草､木の葉､石､野火の煙のにほひ､――さう云ふものが､一つづつ､五位の心に､浮んで来た｡殊に､雀色時の靄の中を､やつと､この館へ辿りついて､長櫃に起してある､炭火の赤い焔を見た時の､ほつとした心もち､――それも､今かうして､寝てゐると､遠い昔にあつた事としか､思はれない｡五位は綿の四五寸もはいつた､黄いろい直垂の下に､楽々と､足をのばしながら､ぼんやり､われとわが寝姿を見廻した｡
 直垂の下に利仁が貸してくれた､練色の衣の綿厚なのを､二枚まで重ねて､着こんでゐる｡それだけでも､どうかすると､汗が出かねない程､暖かい｡そこへ､夕飯の時に一杯やつた､酒の酔が手伝つてゐる｡枕元の蔀一つ隔てた向うは､霜の冴えた広庭だが､それも､かう陶然としてゐれば､少しも苦にならない｡万事が､京都の自分の曹司にゐた時と比べれば､雲泥の相違である｡が､それにも係はらず､我五位の心には､何となく釣合のとれない不安があつた｡第一､時間のたつて行くのが､待遠い｡しかもそれと同時に､夜の明けると云ふ事が､――芋粥を食ふ時になると云ふ事が､さう早く､来てはならないやうな心もちがする｡さうして又､この矛盾した二つの感情が､互に剋し合ふ後には､境遇の急激な変化から来る､落着かない気分が､今日の天気のやうに､うすら寒く控へてゐる｡それが､皆､邪魔になつて､折角の暖かさも､容易に､眠りを誘ひさうもない｡
 すると､外の広庭で､誰か大きな声を出してゐるのが､耳にはいつた｡声がらでは､どうも､今日､途中まで迎へに出た､白髪の郎等が何か告れてゐるらしい｡その乾からびた声が､霜に響くせゐか､凛々として凩のやうに､一語づつ五位の骨に､応へるやうな気さへする｡
｢この辺の下人､承はれ｡殿の御意遊ばさるるには､明朝､卯時までに､切口三寸､長さ五尺の山の芋を､老若各､一筋づつ､持つて参る様にとある｡忘れまいぞ､卯時までにぢや｡｣
 それが､二三度､繰返されたかと思ふと､やがて､人のけはひが止んで､あたりは忽ち元のやうに､静な冬の夜になつた｡その静な中に､切燈台の油が鳴る｡赤い真綿のやうな火が､ゆらゆらする｡五位は欠伸を一つ､噛みつぶして､又､とりとめのない､思量に耽り出した｡――山の芋と云ふからには､勿論芋粥にする気で､持つて来させるのに相違ない｡さう思ふと､一時､外に注意を集中したおかげで忘れてゐた､さつきの不安が､何時の間にか､心に帰つて来る｡殊に､前よりも､一層強くなつたのは､あまり早く芋粥にありつきたくないと云ふ心もちで､それが意地悪く､思量の中心を離れない｡どうもかう容易に｢芋粥に飽かむ｣事が､事実となつて現れては､折角今まで､何年となく､辛抱して待つてゐたのが､如何にも､無駄な骨折のやうに､見えてしまふ｡出来る事なら､突然何か故障が起つて一旦､芋粥が飲めなくなつてから､又､その故障がなくなつて､今度は､やつとこれにありつけると云ふやうな､そんな手続きに､万事を運ばせたい｡――こんな考へが､｢こまつぶり｣のやうに､ぐるぐる一つ所を廻つてゐる中に､何時か､五位は､旅の疲れで､ぐつすり､熟睡してしまつた｡
 翌朝､眼がさめると､直に､昨夜の山の芋の一件が､気になるので､五位は､何よりも先に部屋の蔀をあげて見た｡すると､知らない中に､寝すごして､もう卯時をすぎてゐたのであらう｡広庭へ敷いた､四五枚の長筵の上には､丸太のやうな物が､凡そ､二三千本､斜につき出した､檜皮葺の軒先へつかへる程､山のやうに､積んである｡見るとそれが､悉く､切口三寸､長さ五尺の途方もなく大きい､山の芋であつた｡
 五位は､寝起きの眼をこすりながら､殆ど周章に近い驚愕に襲はれて､呆然と､周囲を見廻した｡広庭の所々には､新しく打つたらしい杭の上に五斛納釜を五つ六つ､かけ連ねて､白い布の襖を着た若い下司女が､何十人となく､そのまはりに動いてゐる｡火を焚きつけるもの､灰を掻くもの､或は､新しい白木の桶に､｢あまづらみせん｣を汲んで釜の中へ入れるもの､皆芋粥をつくる準備で､眼のまはる程忙しい｡釜の下から上る煙と､釜の中から湧く湯気とが､まだ消え残つてゐる明方の靄と一つになつて､広庭一面､はつきり物も見定められない程､灰色のものが罩めた中で､赤いのは､烈々と燃え上る釜の下の焔ばかり､眼に見るもの､耳に聞くもの悉く､戦場か火事場へでも行つたやうな騒ぎである｡五位は､今更のやうに､この巨大な山の芋が､この巨大な五斛納釜の中で､芋粥になる事を考へた｡さうして､自分が､その芋粥を食ふ為に京都から､わざわざ､越前の敦賀まで旅をして来た事を考へた｡考へれば考へる程､何一つ､情無くならないものはない｡我五位の同情すべき食慾は､実に､此時もう､一半を減却してしまつたのである｡
 それから､一時間の後､五位は利仁や舅の有仁と共に､朝飯の膳に向つた｡前にあるのは､銀の提の一斗ばかりはいるのに､なみなみと海の如くたたへた､恐るべき芋粥である｡五位はさつき､あの軒まで積上げた山の芋を､何十人かの若い男が､薄刃を器用に動かしながら､片端から削るやうに､勢よく切るのを見た｡それからそれを､あの下司女たちが､右往左往に馳せちがつて､一つのこらず､五斛納釜へすくつては入れ､すくつては入れするのを見た｡最後に､その山の芋が､一つも長筵の上に見えなくなつた時に､芋のにほひと､甘葛のにほひとを含んだ､幾道かの湯気の柱が､蓬々然として､釜の中から､晴れた朝の空へ､舞上つて行くのを見た｡これを､目のあたりに見た彼が､今､提に入れた芋粥に対した時､まだ､口をつけない中から､既に､満腹を感じたのは､恐らく､無理もない次第であらう｡――五位は､提を前にして､間の悪さうに､額の汗を拭いた｡
｢芋粥に飽かれた事が､ござらぬげな｡どうぞ､遠慮なく召上つて下され｡｣
 舅の有仁は､童児たちに云ひつけて､更に幾つかの銀の提を膳の上に並べさせた｡中にはどれも芋粥が､溢れんばかりにはいつてゐる｡五位は眼をつぶつて､唯でさへ赤い鼻を､一層赤くしながら､提に半分ばかりの芋粥を大きな土器にすくつて､いやいやながら飲み干した｡
｢父も､さう申すぢやて｡平に､遠慮は御無用ぢや｡｣
 利仁も側から､新な提をすすめて､意地悪く笑ひながらこんな事を云ふ｡弱つたのは五位である｡遠慮のない所を云へば､始めから芋粥は､一椀も吸ひたくない｡それを今､我慢して､やつと､提に半分だけ平げた｡これ以上､飲めば､喉を越さない中にもどしてしまふ､さうかと云つて､飲まなければ､利仁や有仁の厚意を無にするのも､同じである｡そこで､彼は又眼をつぶつて､残りの半分を三分の一程飲み干した｡もう後は一口も吸ひやうがない｡
｢何とも､忝うござつた｡もう十分頂戴致したて｡――いやはや､何とも忝うござつた｡｣
 五位は､しどろもどろになつて､かう云つた｡余程弱つたと見えて､口髭にも､鼻の先にも､冬とは思はれない程､汗が玉になつて､垂れてゐる｡
｢これは又､御少食ぢや｡客人は､遠慮をされると見えたぞ｡それそれその方ども､何を致して居る｡｣
 童児たちは､有仁の語につれて､新な提の中から､芋粥を､土器に汲まうとする｡五位は､両手を蠅でも逐ふやうに動かして､平に､辞退の意を示した｡
｢いや､もう､十分でござる｡……失礼ながら､十分でござる｡｣
 もし､此時､利仁が､突然､向うの家の軒を指して､｢あれを御覧じろ｣と云はなかつたなら､有仁は猶､五位に､芋粥をすすめて､止まなかつたかも知れない｡が､幸ひにして､利仁の声は､一同の注意を､その軒の方へ持つて行つた｡檜皮葺の軒には､丁度､朝日がさしてゐる｡さうして､そのまばゆい光に､光沢のいい毛皮を洗はせながら､一疋の獣が､おとなしく､坐つてゐる｡見るとそれは一昨日､利仁が枯野の路で手捕りにした､あの阪本の野狐であつた｡
｢狐も､芋粥が欲しさに､見参したさうな｡男ども､しやつにも､物を食はせてつかはせ｡｣
 利仁の命令は､言下に行はれた｡軒からとび下りた狐は､直に広庭で芋粥の馳走に､与つたのである｡
 五位は､芋粥を飲んでゐる狐を眺めながら､此処へ来ない前の彼自身を､なつかしく､心の中でふり返つた｡それは､多くの侍たちに愚弄されてゐる彼である｡京童にさへ｢何ぢや｡この鼻赤めが｣と､罵られてゐる彼である｡色のさめた水干に､指貫をつけて､飼主のない尨犬のやうに､朱雀大路をうろついて歩く､憐む可き､孤独な彼である｡しかし､同時に又､芋粥に飽きたいと云ふ慾望を､唯一人大事に守つてゐた､幸福な彼である｡――彼は､この上芋粥を飲まずにすむと云ふ安心と共に､満面の汗が次第に､鼻の先から､乾いてゆくのを感じた｡晴れてはゐても､敦賀の朝は､身にしみるやうに､風が寒い｡五位は慌てて､鼻をおさへると同時に銀の提に向つて大きな嚔をした｡"""
    }

]


In [31]:
try:
    result = search_client.upload_documents(documents=documents)
    print("Upload of new document succeeded: {}".format(result[0].succeeded))
except Exception as ex:
    print (ex.message)

Upload of new document succeeded: True


In [32]:
results =  search_client.search(query_type='simple',
    search_text="*" ,
    select='BookName,BookAuthor',
    include_total_count=True)

print ('Total Documents Matching Query:', results.get_count())
for result in results:
    print(result["@search.score"])
    print(result["BookName"])
    print(f"BookAuthor: {result['BookAuthor']}")

Total Documents Matching Query: 5
1.0
鼻
BookAuthor: 芥川龍之介
1.0
蜘蛛の糸
BookAuthor: 芥川龍之介
1.0
羅生門
BookAuthor: 芥川龍之介
1.0
河童
BookAuthor: 芥川龍之介
1.0
芋粥
BookAuthor: 芥川龍之介


In [33]:
results =  search_client.search(query_type='simple',
    search_text="What book did Ryunosuke Akutagawa write?" ,
    select='BookName,BookAuthor,Rating')

for result in results:
    print(result["@search.score"])
    print(result["BookName"])
    print(f"Text: {result['Text']}")

In [34]:
results =  search_client.search(query_type='semantic', semantic_configuration_name='my-semantic-config',
    search_text="What books are highly rated", 
    select='BookName,BookAuthor,Rating', query_caption='extractive')

for result in results:
    print(result["@search.reranker_score"])
    print(result["BookName"])
    print(f"Text: {result['Text']}")

    captions = result["@search.captions"]
    if captions:
        caption = captions[0]
        if caption.highlights:
            print(f"Caption: {caption.highlights}\n")
        else:
            print(f"Caption: {caption.text}\n")

In [36]:
results =  search_client.search(query_type='semantic', query_language='en-us', semantic_configuration_name='my-semantic-config',
 search_text="What books are highly rated?", 
 select='BookName,BookAuthor,Rating', query_caption='extractive', query_answer="extractive",)

for result in results:
    print(result["@search.reranker_score"])
    print(result["BookName"])
    print(f"Text: {result['Text']}")

    captions = result["@search.captions"]
    if captions:
        caption = captions[0]
        if caption.highlights:
            print(f"Caption: {caption.highlights}\n")
        else:
            print(f"Caption: {caption.text}\n")